In [43]:
%load_ext autoreload
%autoreload 2

import spglib as spg
import ase
from ase.visualize import view

import ase.spacegroup as aspg
import numpy as np
import pandas as pd
from collections import Iterable
import cPickle as pck
import spglib as spg
import sys, os
# add to PYTHONPATH
#modulepath = '/home/musil/Dropbox/Felix_work/qmat/spglib-test/'
#modulepath = '/Users/iMac/Dropbox/Felix_work/qmat/spglib-test/'
#if modulepath not in sys.path:
#    sys.path.insert(0,modulepath)
#from genGeneralSites import GenCrystalStructures
from utils import rotation_matrix,isCellSkewed,unskewCell,get_symprec,get_relative_angle
from qe_input import makeCard,makeNamelist,SG2ibrav,SG2wyckoff,makeQEInput_sg,makeQEInput_ibrav0,makeQEInput

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Utility functions

# import mined info on SG, Wickoff, elements and some structures

In [13]:
fileNames = {}
infoPath = '../info/'
structurePath = '../structures/'
fileNames['crystals'] = structurePath + 'partial_input_crystals_sg3-230.pck'
fileNames['wyck'] = infoPath+'SpaceGroup-multiplicity-wickoff-info.pck'
fileNames['general info'] = infoPath+'SpaceGroup-general-info.pck'
fileNames['elements info'] = infoPath+'General-Info-Elements-fast.pck'

In [14]:
with open(fileNames['crystals'],'rb') as f:
    crystals = pck.load(f)
with open(fileNames['wyck'],'rb') as f:
    WyckTable = pck.load(f)
SGTable = pd.read_pickle(fileNames['general info'])
ElemTable = pd.read_pickle(fileNames['elements info'])

In [63]:
from custom_frame import frame2qe_format
sg = 68
cc = crystals[sg][3]
dd = frame2qe_format(cc,sg)
view(cc)
view(dd)

In [65]:
inpt = makeQEInput(cc,sg,WyckTable,SGTable,ElemTable,
                 zatom = 14,rhocutoff = 20 * 4,wfccutoff = 20,
                 calculation_type='"scf"',smearing=1e-2,
                 kpt = [2,2,2],Nkpt=1000,kpt_offset = [0,0,0],ppPath='"./pseudo/"',
                 PP=['Si.pbe-n-rrkjus_psl.1.0.0.UPF'])
print inpt

&CONTROL 
  calculation = "scf" 
  outdir = "./out/" 
  prefix = "qe" 
  pseudo_dir = "./pseudo/" 
  restart_mode = "from_scratch" 
  verbosity = "high" 
  wf_collect = .false. 
  nstep = 150 
  etot_conv_thr = 0.00080 
  forc_conv_thr = 0.00100 
/ 
&SYSTEM 
  ecutrho = 80.00000 
  ecutwfc = 20.00000 
  ibrav = -9 
  nat = 1 
  nbnd = 20 
  ntyp = 1 
  occupations = "smearing" 
  smearing = "cold" 
  degauss = 0.010000 
  space_group = 68 
  uniqueb = .FALSE. 
  A = 4.18398277665 
  B = 4.18398277665 
  C = 6.17850824706 
  cosAB = -0.70628 
  cosAC = 0.00000 
  cosBC = 0.00000 
/ 
&ELECTRONS 
  conv_thr = 1.0000000000d-06 
/ 
&CELL 
  press = 0.00000 
  press_conv_thr = 0.50000 
  cell_factor = 2.00000 
/ 
ATOMIC_SPECIES   
  Si  28.0855  Si.pbe-n-rrkjus_psl.1.0.0.UPF 
ATOMIC_POSITIONS  crystal_sg 
  Si  16i   0.14233355  0.35761389  0.62499229 
K_POINTS  automatic 
  5  6  3  0  0  0 



## test

In [10]:
SGTable.head(5)

,Serial No.,Table No.,Sub No.,"Main axis, Setting",Hall symbol,SF symbol,HM symbol,Full notation,No. of Point group,Point group (HM),Point group (SF),No. of Laue Group,Laue Group,No. of Crystal System,Crystal System
1,1,1,1,NaN,P 1,C11,P 1,P 1,1,1,C1,1,-1,1,triclinic
2,2,2,1,NaN,-P 1,Ci1,P -1,P -1,2,-1,Ci,1,-1,1,triclinic
3,3,3,1,b,P 2y,C21,P 2 = P 1 2 1,P 1 2 1,3,2,C2,2,2/m,2,monoclinic
4,6,4,1,b,P 2yb,C22,P 21 = P 1 21 1,P 1 21 1,3,2,C2,2,2/m,2,monoclinic
5,9,5,1,b1,C 2y,C23,C 2 = C 1 2 1,C 1 2 1,3,2,C2,2,2/m,2,monoclinic


In [11]:
df = SGTable.rename(columns = {'Table No.':'a'})
SGTable[SGTable['Table No.'] == 42]

,Serial No.,Table No.,Sub No.,"Main axis, Setting",Hall symbol,SF symbol,HM symbol,Full notation,No. of Point group,Point group (HM),Point group (SF),No. of Laue Group,Laue Group,No. of Crystal System,Crystal System
42,209,42,1,NaN,F 2 -2,C2v18,F m m 2,F m m 2,7,m m 2,C2v,3,mmm,3,orthorhombic


# Test how qe cells definitions

In [6]:
from ase.spacegroup import crystal

In [7]:
#crystal(symbols=None, basis=None, spacegroup=1, setting=1, cell=None, cellpar=None, ab_normal=(0, 0, 1), a_direction=None, size=(1, 1, 1), onduplicates='warn', symprec=0.001, pbc=True, primitive_cell=False, **kwargs)

In [69]:
sg2ibrav = {it:SG2ibrav(it,SGTable) for it in range(1,230+1)}
print sg2ibrav

{1: 14, 2: 14, 3: -12, 4: -12, 5: 13, 6: -12, 7: -12, 8: 13, 9: 13, 10: -12, 11: -12, 12: 13, 13: -12, 14: -12, 15: 13, 16: 8, 17: 8, 18: 8, 19: 8, 20: -9, 21: -9, 22: 10, 23: 11, 24: 11, 25: 8, 26: 8, 27: 8, 28: 8, 29: 8, 30: 8, 31: 8, 32: 8, 33: 8, 34: 8, 35: -9, 36: -9, 37: -9, 38: -9, 39: -9, 40: -9, 41: -9, 42: 10, 43: 10, 44: 11, 45: 11, 46: 11, 47: 8, 48: 8, 49: 8, 50: 8, 51: 8, 52: 8, 53: 8, 54: 8, 55: 8, 56: 8, 57: 8, 58: 8, 59: 8, 60: 8, 61: 8, 62: 8, 63: -9, 64: -9, 65: -9, 66: -9, 67: -9, 68: -9, 69: 10, 70: 10, 71: 11, 72: 11, 73: 11, 74: 11, 75: 6, 76: 6, 77: 6, 78: 6, 79: 7, 80: 7, 81: 6, 82: 7, 83: 6, 84: 6, 85: 6, 86: 6, 87: 7, 88: 7, 89: 6, 90: 6, 91: 6, 92: 6, 93: 6, 94: 6, 95: 6, 96: 6, 97: 7, 98: 7, 99: 6, 100: 6, 101: 6, 102: 6, 103: 6, 104: 6, 105: 6, 106: 6, 107: 7, 108: 7, 109: 7, 110: 7, 111: 6, 112: 6, 113: 6, 114: 6, 115: 6, 116: 6, 117: 6, 118: 6, 119: 7, 120: 7, 121: 7, 122: 7, 123: 6, 124: 6, 125: 6, 126: 6, 127: 6, 128: 6, 129: 6, 130: 6, 131: 6, 132: 6,

## ibrav 1 
 1          cubic P (sc)
      v1 = a(1,0,0),  v2 = a(0,1,0),  v3 = a(0,0,1)
      
NO PROPBLEM

In [9]:
relevant_sg = []
for key,item in sg2ibrav.iteritems():
    if item == 1:
        relevant_sg.append(key)
print relevant_sg

[195, 198, 200, 201, 205, 207, 208, 212, 213, 215, 218, 221, 222, 223, 224]


In [51]:
sg = 223
cc = crystals[sg][0]
print cc.get_cell()
print cc.get_cell_lengths_and_angles()
Na = cc.get_number_of_atoms()
pos = cc.get_scaled_positions()[0]
cell_par = cc.get_cell_lengths_and_angles()
view(cc)

[[  1.10448729e+01  -3.62415604e-17  -1.00366923e-17]
 [ -3.62415604e-17   1.10448729e+01   6.16575191e-17]
 [ -1.00366923e-17   6.16575191e-17   1.10448729e+01]]
[ 11.04487288  11.04487288  11.04487288  90.          90.          90.        ]


In [52]:
a,b,c,alpha,beta,gamma =  cell_par
v1 = [a,0,0]
v2 = [0,a,0]
v3 = [0,0,a]
vv = np.array([v1,v2,v3])
dd = crystal(basis = pos,symbols=['Si',]*Na,  spacegroup=sg,  cell=vv)
view(dd)

## ibrav 2
2          cubic F (fcc)
      v1 = (a/2)(-1,0,1),  v2 = (a/2)(0,1,1), v3 = (a/2)(-1,1,0)
      
go to primitive cell

In [61]:
a = 17.32926009;
v1 = a/2*np.array([-1,0,1]);   
v2 = a/2*np.array([0,1,1]);  
v3 = a/2*np.array([-1,1,0])
print np.array([v1,v2,v3])

[[-8.66463004  0.          8.66463004]
 [ 0.          8.66463004  8.66463004]
 [-8.66463004  8.66463004  0.        ]]


### investigate the prob

In [62]:
sg = 225
cc = crystals[sg][0]
print cc.get_cell()
print cc.get_cell_lengths_and_angles()
Na = cc.get_number_of_atoms()
pos = cc.get_scaled_positions()[0]
cell_par = cc.get_cell_lengths_and_angles()
view(cc)

[[  1.73292601e+01  -5.78623416e-17  -6.65376850e-17]
 [ -5.78623416e-17   1.73292601e+01   8.13454945e-17]
 [ -6.65376850e-17   8.13454945e-17   1.73292601e+01]]
[ 17.32926009  17.32926009  17.32926009  90.          90.          90.        ]


In [57]:
b1,b2,b3 = np.array([[  1.73292601e+01,  0.,  0.],
                     [ 0.,   1.73292601e+01,   0.],
                     [0.,   0.,   1.73292601e+01]])
nb1 = (-b1+b3)/2
nb2 = ( b2+b3)/2
nb3 = (-b1+b2)/2

nb = np.array([nb1,nb2,nb3])

[-8.66463005  0.          8.66463005]


In [63]:
import spglib as spg
print spg.get_spacegroup(cc)
(lattice, positions, numbers) = spg.find_primitive(cc, symprec=1e-05, angle_tolerance=-1.0)
ee = ase.Atoms(cell=lattice,scaled_positions=positions,numbers=numbers)
print ee.get_cell()
view(ee)


Fm-3m (225)
[[ 0.          8.66463004  8.66463004]
 [ 8.66463004  0.          8.66463004]
 [ 8.66463004  8.66463004  0.        ]]


In [64]:
sg = 225
cc = crystals[sg][0]
# view(cc)
(lattice, positions, numbers) = spg.find_primitive(cc, symprec=1e-05, angle_tolerance=-1.0)
ee = ase.Atoms(cell=lattice,scaled_positions=positions,numbers=numbers)
# view(ee)
cell = ee.get_cell()
pos = ee.get_positions()
v2 = cell[0]
axis = [0,0,1]
theta = np.pi / 2.
rot = rotation_matrix(axis, theta)
v2p = np.dot(rot,v2)
cellp = np.dot(cell,rot.T)
posp = np.dot(pos,rot.T)
print cell
print v2p
print cellp
print pos[0],posp[0]

cellp[np.abs(cellp)<1e-10] = 0.
ee.set_cell(cellp)
ee.set_positions(posp)
a = cellp[cellp>0][0] * 2
# view(ee)
print ee.get_cell()
print ee.get_cell_lengths_and_angles()
print a

[[ 0.          8.66463004  8.66463004]
 [ 8.66463004  0.          8.66463004]
 [ 8.66463004  8.66463004  0.        ]]
[ -8.66463004e+00   1.92393435e-15   8.66463004e+00]
[[ -8.66463004e+00   1.92393435e-15   8.66463004e+00]
 [  1.92393435e-15   8.66463004e+00   8.66463004e+00]
 [ -8.66463004e+00   8.66463004e+00   0.00000000e+00]]
[ 5.92936369  4.33202087  9.79639299] [-4.33202087  5.92936369  9.79639299]
[[-8.66463004  0.          8.66463004]
 [ 0.          8.66463004  8.66463004]
 [-8.66463004  8.66463004  0.        ]]
[ 12.25363732  12.25363732  12.25363732  60.          60.          60.        ]
17.329260085


### functions to solve it

In [65]:
def get_ibrav2_atoms(frame,sg):
    import spglib as spg
    symprec = get_symprec(frame,sg)
    if symprec is None:
        print 'Not possible'
        return None
    (lattice, positions, numbers) = spg.standardize_cell(frame, to_primitive=True, no_idealize=False, symprec=symprec, angle_tolerance=-1.0)
    primitive_atoms = ase.Atoms(cell=lattice,scaled_positions=positions,numbers=numbers)

    cell = primitive_atoms.get_cell()
    pos = primitive_atoms.get_positions()
    
    axis = [0,0,1]
    theta = np.pi / 2.
    rot = rotation_matrix(axis, theta)
    
    cellp = np.dot(cell,rot.T)
    cellp[np.abs(cellp)<1e-10] = 0.
    posp = np.dot(pos,rot.T)
    inequivalent_pos = posp[0]
    
    primitive_atoms.set_cell(cellp)
    primitive_atoms.set_positions(posp)
    
    a = primitive_atoms.get_cell_lengths_and_angles()[0] * 2
    # only a is used by QE
    b,c,alpha,beta,gamma = [0.]*5
    cell_par = [a,b,c,alpha,beta,gamma]
    return primitive_atoms,cell_par,inequivalent_pos

### check all relevant sg

In [22]:
TOMODIFY = [5, 8, 9, 12, 15,23, 24, 44, 45, 46, 71, 72, 73, 74,22, 42, 43,
          69, 70,79, 80, 82,87, 88, 97, 98, 107, 108, 109,110, 119, 120, 121, 122,
          139, 140, 141, 142,197, 199, 204, 206, 211, 214, 217, 220, 229, 230,
            196, 202, 203, 209, 210, 216, 219, 225, 226, 227, 228,20, 21, 35, 36, 37, 38, 39, 40, 
            41, 63, 64, 65, 66, 67, 68, 3, 4, 6, 7, 10, 11, 13, 14,146, 148, 155, 160, 161, 166, 167,
           79, 80, 82, 87, 88, 97, 98, 107, 108, 109, 110, 119, 120, 121, 122, 139, 140, 141, 142]
print sorted(TOMODIFY)

[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 20, 21, 22, 23, 24, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 79, 79, 80, 80, 82, 82, 87, 87, 88, 88, 97, 97, 98, 98, 107, 107, 108, 108, 109, 109, 110, 110, 119, 119, 120, 120, 121, 121, 122, 122, 139, 139, 140, 140, 141, 141, 142, 142, 146, 148, 155, 160, 161, 166, 167, 196, 197, 199, 202, 203, 204, 206, 209, 210, 211, 214, 216, 217, 219, 220, 225, 226, 227, 228, 229, 230]


In [26]:
relevant_sg = []
for key,item in sg2ibrav.iteritems():
    if item == 2:
        relevant_sg.append(key)
print relevant_sg

[196, 202, 203, 209, 210, 216, 219, 225, 226, 227, 228]


In [34]:
sg = 203
cc = crystals[sg][0]
primitive_atoms,cell_par,inequivalent_pos = get_ibrav2_atoms(cc)
view(cc)
view(primitive_atoms)
print 'sg {}'.format(sg)
print cell_par


sg 203
[10.928800830147843, 0.0, 0.0, 0.0, 0.0, 0.0]


In [35]:
symprec = 1e-5
print spg.get_spacegroup(cc,symprec=symprec)
print spg.get_spacegroup(primitive_atoms,symprec=symprec)

Fd3 (203)
Fd3 (203)


In [11]:
for sg in relevant_sg:
    try:
        cc = crystals[sg][0]
        primitive_atoms,cell_par,inequivalent_pos = get_ibrav2_atoms(cc)
        view(cc)
        view(primitive_atoms)
        print 'sg {}'.format(sg)
        print cell_par
        raw_input("Press Enter to continue...")
    except:
        print 'No structure for sg {}'.format(sg)



No structure for sg 196
No structure for sg 202
No structure for sg 203
No structure for sg 209
No structure for sg 210
No structure for sg 216
No structure for sg 219
No structure for sg 225
No structure for sg 226
No structure for sg 227
No structure for sg 228


## ibrav 3
3          cubic I (bcc)
      v1 = (a/2)(1,1,1),  v2 = (a/2)(-1,1,1),  v3 = (a/2)(-1,-1,1)
      
PROBLEM --> ibrav0

https://www.mail-archive.com/pw_forum@pwscf.org/msg30629.html

The cell definition is not the standard one and it's hard to tranform the cell as expected by QE
---------> fall back to ibrav 0

In [14]:
relevant_sg = []
for key,item in sg2ibrav.iteritems():
    if item == 3:
        relevant_sg.append(key)
print relevant_sg

[197, 199, 204, 206, 211, 214, 217, 220, 229, 230]


In [15]:
sg = 230
cc = crystals[sg][0]
print cc.get_cell()
print cc.get_cell_lengths_and_angles()
Na = cc.get_number_of_atoms()
pos = cc.get_scaled_positions()[0]
cell_par = cc.get_cell_lengths_and_angles()
view(cc)

[[  1.17301762e+01  -5.96588968e-15   3.76947835e-16]
 [ -5.96588968e-15   1.17301762e+01   1.76323546e-16]
 [  3.76947835e-16   1.76323546e-16   1.17301762e+01]]
[ 11.73017616  11.73017616  11.73017616  90.          90.          90.        ]


In [38]:
sg = 230
cc = crystals[sg][0]
# view(cc)
(lattice, positions, numbers) = spg.find_primitive(cc, symprec=1e-05, angle_tolerance=-1.0)
ee = ase.Atoms(cell=lattice,scaled_positions=positions,numbers=numbers)
view(ee)
cell = ee.get_cell()
pos = ee.get_positions()
v2 = cell[0]
axis = [0,0,1]
theta = np.pi / 2.
rot = rotation_matrix(axis, theta)
v2p = np.dot(rot,v2)
cellp = np.dot(cell,rot.T)
posp = np.dot(pos,rot.T)
print cell
print v2p
print cellp
print pos[0],posp[0]

cellp[np.abs(cellp)<1e-10] = 0.
ee.set_cell(cellp)
ee.set_positions(posp)
a = cellp[cellp>0][0] * 2
# view(ee)
print ee.get_cell()
print ee.get_cell_lengths_and_angles()
print a

[[-5.86508808  5.86508808  5.86508808]
 [ 5.86508808 -5.86508808  5.86508808]
 [ 5.86508808  5.86508808 -5.86508808]]
[-5.86508808 -5.86508808  5.86508808]
[[-5.86508808 -5.86508808  5.86508808]
 [ 5.86508808  5.86508808  5.86508808]
 [-5.86508808  5.86508808 -5.86508808]]
[ 2.36019451  1.04105889  8.43949339] [-1.04105889  2.36019451  8.43949339]
[[-5.86508808 -5.86508808  5.86508808]
 [ 5.86508808  5.86508808  5.86508808]
 [-5.86508808  5.86508808 -5.86508808]]
[  10.15863055   10.15863055   10.15863055  109.47122063  109.47122063
  109.47122063]
11.7301761627


In [16]:
def get_ibrav3_atoms(frame,sg):
    import spglib as spg
    symprec = get_symprec(frame,sg)
    if symprec is None:
        print 'Not possible'
        return None
    (lattice, positions, numbers) = spg.standardize_cell(frame, to_primitive=True, no_idealize=False, symprec=symprec, angle_tolerance=-1.0)
    primitive_atoms = ase.Atoms(cell=lattice,scaled_positions=positions,numbers=numbers)

    cell = primitive_atoms.get_cell()
    pos = primitive_atoms.get_positions()
    
    inequivalent_pos = pos[0]
    
    return primitive_atoms,cell,inequivalent_pos

### check all

In [17]:
sg = 230
cc = crystals[sg][0]
primitive_atoms,cell_par,inequivalent_pos = get_ibrav3_atoms(cc,sg)
view(cc)
view(primitive_atoms)
print 'sg {}'.format(sg)
print cell_par


sg 230
[[ 5.86508808  5.86508808 -5.86508808]
 [ 5.86508808 -5.86508808  5.86508808]
 [-5.86508808 -5.86508808 -5.86508808]]


In [18]:
symprec = 1e-5
print spg.get_spacegroup(cc,symprec=symprec)
print spg.get_spacegroup(primitive_atoms,symprec=symprec)

Ia-3d (230)
Ia-3d (230)


In [19]:
for sg in relevant_sg:
    try:
        cc = crystals[sg][0]
        primitive_atoms,cell_par,inequivalent_pos = get_ibrav3_atoms(cc,sg)
        view(cc)
        view(primitive_atoms)
        print 'sg {}'.format(sg)
        print cell_par
        raw_input("Press Enter to continue...")
    except:
        print 'No structure for sg {}'.format(sg)



sg 197
[[ 3.19117421  3.19117421  3.19117421]
 [ 3.19117421 -3.19117421 -3.19117421]
 [-3.19117421  3.19117421 -3.19117421]]
Press Enter to continue...
sg 199
[[ 3.18175146  3.18175146  3.18175146]
 [ 3.18175146 -3.18175146 -3.18175146]
 [-3.18175146  3.18175146 -3.18175146]]
Press Enter to continue...
sg 204
[[-4.42122989 -4.42122989  4.42122989]
 [-4.42122989  4.42122989 -4.42122989]
 [ 4.42122989 -4.42122989 -4.42122989]]
Press Enter to continue...
sg 206
[[-3.81999479 -3.81999479  3.81999479]
 [-3.81999479  3.81999479 -3.81999479]
 [ 3.81999479 -3.81999479 -3.81999479]]
Press Enter to continue...
sg 211
[[ 4.40150515 -4.40150515  4.40150515]
 [ 4.40150515  4.40150515 -4.40150515]
 [-4.40150515  4.40150515  4.40150515]]
Press Enter to continue...
sg 214
[[ 4.7688406  4.7688406 -4.7688406]
 [ 4.7688406 -4.7688406  4.7688406]
 [-4.7688406 -4.7688406 -4.7688406]]
Press Enter to continue...
sg 217
[[-4.44131934 -4.44131934  4.44131934]
 [-4.44131934  4.44131934 -4.44131934]
 [ 4.4413193

## ibrav 4
4          Hexagonal and Trigonal P        celldm(3)=c/a
      v1 = a(1,0,0),  v2 = a(-1/2,sqrt(3)/2,0),  v3 = a(0,0,c/a)
      
      
NO PROBLEM 

In [39]:
relevant_sg = []
for key,item in sg2ibrav.iteritems():
    if item == 4:
        relevant_sg.append(key)
print relevant_sg

[143, 144, 145, 147, 149, 150, 151, 152, 153, 154, 156, 157, 158, 159, 162, 163, 164, 165, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194]


In [40]:
sg = 192
cc = crystals[sg][0]
print cc.get_cell()
print cc.get_cell_lengths_and_angles()
Na = cc.get_number_of_atoms()
pos = cc.get_scaled_positions()[0]
cell_par = cc.get_cell_lengths_and_angles()
view(cc)

[[  1.05967022e+01   4.94275755e-16   1.38167505e-13]
 [ -5.29835111e+00   9.17701332e+00   1.94072597e-13]
 [  5.71181187e-14   1.25905178e-13   4.48680887e+00]]
[  10.59670222   10.59670222    4.48680887   90.           90.          120.        ]


## ibrav 5
5          Trigonal R, 3fold axis c        celldm(4)=cos(alpha)
      The crystallographic vectors form a three-fold star around
      the z-axis, the primitive cell is a simple rhombohedron:
      
      v1 = a(tx,-ty,tz),   v2 = a(0,2ty,tz),   v3 = a(-tx,-ty,tz)
      
      where c=cos(alpha) is the cosine of the angle alpha between
      any pair of crystallographic vectors, tx, ty, tz are:
      
        tx=sqrt((1-c)/2), ty=sqrt((1-c)/6), tz=sqrt((1+2c)/3)
        
PROBLEM -> go to primitive cell

In [139]:
a = 4.42371595; c = np.cos(109.192587*np.pi/180.)
tx = np.sqrt((1-c)/2); ty = np.sqrt((1-c)/6); tz = np.sqrt((1+2*c)/3)
v1 = a*np.array([tx,-ty,tz]);   v2 = a*np.array([0,2*ty,tz]);   v3 = a*np.array([-tx,-ty,tz])
print np.array([v1,v2,v3])

[[ 3.60572805 -2.08176806  1.49473402]
 [ 0.          4.16353612  1.49473402]
 [-3.60572805 -2.08176806  1.49473402]]


In [44]:
relevant_sg = []
for key,item in sg2ibrav.iteritems():
    if item == 5:
        relevant_sg.append(key)
print relevant_sg

[146, 148, 155, 160, 161, 166, 167]


### investigate the prob

In [45]:
sg = 155
cc = crystals[sg][0]
print cc.get_cell()
print cc.get_cell_lengths_and_angles()
Na = cc.get_number_of_atoms()
pos = cc.get_scaled_positions()[0]
cell_par = cc.get_cell_lengths_and_angles()
view(cc)

[[  7.21145610e+00   1.82300988e-14  -2.84013999e-14]
 [ -3.60572805e+00   6.24530418e+00  -2.41148163e-15]
 [ -1.14709818e-14  -7.90864707e-15   4.48420206e+00]]
[   7.2114561     7.2114561     4.48420206   90.           90.          120.        ]


In [111]:
sg = 155
cc = crystals[sg][0]
(lattice, positions, numbers) = spg.find_primitive(cc, symprec=1e-05, angle_tolerance=-1.0)
ee = ase.Atoms(cell=lattice,scaled_positions=positions,numbers=numbers)
cell = ee.get_cell()
pos = ee.get_positions()
v2 = cell[0]
axis = [0,0,1]
theta = - 60*np.pi/180.
rot = rotation_matrix(axis, theta)
v2p = np.dot(rot,v2)
cellp = np.dot(cell,rot.T)
posp = np.dot(pos,rot.T)
print cell
print cellp

[[ 3.60572805  2.08176806  1.49473402]
 [-3.60572805  2.08176806  1.49473402]
 [ 0.         -4.16353612  1.49473402]]
[ 3.60572805 -2.08176806  1.49473402]
[[  3.60572805e+00  -2.08176806e+00   1.49473402e+00]
 [ -1.11022302e-15   4.16353612e+00   1.49473402e+00]
 [ -3.60572805e+00  -2.08176806e+00   1.49473402e+00]]


In [142]:
cellp[np.abs(cellp)<1e-10] = 0.
ee.set_cell(cellp)
ee.set_positions(posp)
a = ee.get_cell_lengths_and_angles()[0]
c = np.cos(ee.get_cell_lengths_and_angles()[3]*np.pi/180.)
view(ee)
print ee.get_cell()
print ee.get_cell_lengths_and_angles()
print a,c

[[ 3.60572805 -2.08176806  1.49473402]
 [ 0.          4.16353612  1.49473402]
 [-3.60572805 -2.08176806  1.49473402]]
[   4.42371595    4.42371595    4.42371595  109.192587    109.192587
  109.192587  ]
4.42371595141 -0.328744459329


### functions to solve it

In [67]:
def get_ibrav5_atoms(frame,sg):
    import spglib as spg
    symprec = get_symprec(frame,sg)
    if symprec is None:
        print 'Not possible'
        return None
    (lattice, positions, numbers) = spg.find_primitive(frame, symprec=symprec, angle_tolerance=-1.0)
    primitive_atoms = ase.Atoms(cell=lattice,scaled_positions=positions,numbers=numbers)

    cell = primitive_atoms.get_cell()
    pos = primitive_atoms.get_positions()
    
    axis = [0,0,1]
    theta = - 60.*np.pi/180.
    rot = rotation_matrix(axis, theta)
    
    cellp = np.dot(cell,rot.T)
    cellp[np.abs(cellp)<1e-10] = 0.
    posp = np.dot(pos,rot.T)
    inequivalent_pos = posp[0]
    
    primitive_atoms.set_cell(cellp)
    primitive_atoms.set_positions(posp)
    
    a,_,_,alpha,_,_ = primitive_atoms.get_cell_lengths_and_angles()
    # only a is used by QE
    b,c,beta,gamma = [0.]*4
    cell_par = [a,b,c,alpha,beta,gamma]
    return primitive_atoms,cell_par,inequivalent_pos

### check all

In [153]:
sg = 167
cc = crystals[sg][0]
primitive_atoms,cell_par,inequivalent_pos = get_ibrav5_atoms(cc,sg)
view(cc)
view(primitive_atoms)
print 'sg {}'.format(sg)
print cell_par


sg 167
[9.7049669483204024, 0.0, 0.0, 119.39306125170683, 0.0, 0.0]


In [154]:
symprec = 1e-5
print spg.get_spacegroup(cc,symprec=symprec)
print spg.get_spacegroup(primitive_atoms,symprec=symprec)

R-3c (167)
R-3c (167)


In [147]:
for sg in relevant_sg:
    try:
        cc = crystals[sg][0]
        primitive_atoms,cell_par,inequivalent_pos = get_ibrav5_atoms(cc,sg)
        view(cc)
        view(primitive_atoms)
        print 'sg {}'.format(sg)
        print cell_par
        raw_input("Press Enter to continue...")
    except:
        print 'No structure for sg {}'.format(sg)



sg 146
[3.7820019948990224, 0.0, 0.0, 0.0, 0.0, 0.0]
Press Enter to continue...
sg 148
[5.4039213269642969, 0.0, 0.0, 0.0, 0.0, 0.0]
Press Enter to continue...
sg 155
[4.4237159514123023, 0.0, 0.0, 0.0, 0.0, 0.0]
No structure for sg 155
sg 160
[5.9663080782218865, 0.0, 0.0, 0.0, 0.0, 0.0]
No structure for sg 160
sg 161
[5.4425698931175646, 0.0, 0.0, 0.0, 0.0, 0.0]
No structure for sg 161
sg 166
[10.209972340390793, 0.0, 0.0, 0.0, 0.0, 0.0]
No structure for sg 166
sg 167
[9.7049669483204024, 0.0, 0.0, 0.0, 0.0, 0.0]
No structure for sg 167


## ibrav 6
6          Tetragonal P (st)               celldm(3)=c/a
      v1 = a(1,0,0),  v2 = a(0,1,0),  v3 = a(0,0,c/a)
   
NO PROBLEM

In [155]:
relevant_sg = []
for key,item in sg2ibrav.iteritems():
    if item == 6:
        relevant_sg.append(key)
print relevant_sg

[75, 76, 77, 78, 81, 83, 84, 85, 86, 89, 90, 91, 92, 93, 94, 95, 96, 99, 100, 101, 102, 103, 104, 105, 106, 111, 112, 113, 114, 115, 116, 117, 118, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138]


### investigate the prob

In [157]:
sg = 137
cc = crystals[sg][0]
print cc.get_cell()
print cc.get_cell_lengths_and_angles()
Na = cc.get_number_of_atoms()
pos = cc.get_scaled_positions()[0]
cell_par = cc.get_cell_lengths_and_angles()
view(cc)

[[  7.25210202e+00   2.74015573e-17   1.01732291e-15]
 [  2.74015573e-17   7.25210202e+00  -6.05728061e-17]
 [  6.16697386e-16  -4.20613289e-17   4.51294622e+00]]
[  7.25210202   7.25210202   4.51294622  90.          90.          90.        ]


## ibrav 7
7          Tetragonal I (bct)              
celldm(3)=c/a

      v1=(a/2)(1,-1,c/a),  v2=(a/2)(1,1,c/a),  v3=(a/2)(-1,-1,c/a)
      
NOT THE STANDARD PRIMITIVE CELL DEFINITION --> ibrav0

In [8]:
a = 9.81743949; c = 2.27955556
v1 = a/2*np.array([1,-1,c/a]);   v2 = a/2*np.array([1,1,c/a]);  
v3 = a/2*np.array([-1,-1,c/a])
print np.array([v1,v2,v3])

[[ 4.90871974 -4.90871974  1.13977778]
 [ 4.90871974  4.90871974  1.13977778]
 [-4.90871974 -4.90871974  1.13977778]]


In [9]:
relevant_sg = []
for key,item in sg2ibrav.iteritems():
    if item == 7:
        relevant_sg.append(key)
print relevant_sg

[79, 80, 82, 87, 88, 97, 98, 107, 108, 109, 110, 119, 120, 121, 122, 139, 140, 141, 142]


### investigate the prob

In [161]:
sg = 97
cc = crystals[sg][0]
print cc.get_cell()
print cc.get_cell_lengths_and_angles()
Na = cc.get_number_of_atoms()
pos = cc.get_scaled_positions()[0]
cell_par = cc.get_cell_lengths_and_angles()
view(cc)

[[  9.81743949e+00   2.49979924e-16   7.59494391e-14]
 [  2.49979924e-16   9.81743949e+00   1.05824960e-13]
 [  1.88313871e-14   2.62365279e-14   2.27955556e+00]]
[  9.81743949   9.81743949   2.27955556  90.          90.          90.        ]


In [168]:
sg = 97
cc = crystals[sg][0]
(lattice, positions, numbers) = spg.find_primitive(cc, symprec=1e-05, angle_tolerance=-1.0)
ee = ase.Atoms(cell=lattice,scaled_positions=positions,numbers=numbers)
cell = ee.get_cell()
pos = ee.get_positions()
v2 = cell[0]
axis = [0,0,1]
theta = - 90*np.pi/180.
rot = rotation_matrix(axis, theta)
v2p = np.dot(rot,v2)
cellp = np.dot(cell,rot.T)
posp = np.dot(pos,rot.T)
print cell
print cellp

[[-4.90871974  4.90871974  1.13977778]
 [ 4.90871974 -4.90871974  1.13977778]
 [ 4.90871974  4.90871974 -1.13977778]]
[[ 4.90871974  4.90871974  1.13977778]
 [-4.90871974 -4.90871974  1.13977778]
 [ 4.90871974 -4.90871974 -1.13977778]]


In [169]:
a = 9.81743949; c = 2.27955556
v1 = a/2*np.array([1,-1,c/a]);   v2 = a/2*np.array([1,1,c/a]);  
v3 = a/2*np.array([-1,-1,c/a])
print np.array([v1,v2,v3])

[[ 4.90871974 -4.90871974  1.13977778]
 [ 4.90871974  4.90871974  1.13977778]
 [-4.90871974 -4.90871974  1.13977778]]


In [142]:
cellp[np.abs(cellp)<1e-10] = 0.
ee.set_cell(cellp)
ee.set_positions(posp)
a = ee.get_cell_lengths_and_angles()[0]
c = np.cos(ee.get_cell_lengths_and_angles()[3]*np.pi/180.)
view(ee)
print ee.get_cell()
print ee.get_cell_lengths_and_angles()
print a,c

[[ 3.60572805 -2.08176806  1.49473402]
 [ 0.          4.16353612  1.49473402]
 [-3.60572805 -2.08176806  1.49473402]]
[   4.42371595    4.42371595    4.42371595  109.192587    109.192587
  109.192587  ]
4.42371595141 -0.328744459329


### functions to solve it

In [151]:
def get_ibrav7_atoms(frame,sg):
    import spglib as spg
    symprec = get_symprec(frame,sg)
    if symprec is None:
        print 'Not possible'
        return None
    (lattice, positions, numbers) = spg.find_primitive(frame, symprec=symprec, angle_tolerance=-1.0)
    primitive_atoms = ase.Atoms(cell=lattice,scaled_positions=positions,numbers=numbers)

    cell = primitive_atoms.get_cell()
    pos = primitive_atoms.get_positions()
    
    
    inequivalent_pos = pos[0]
    
    primitive_atoms.set_cell(cellp)
    primitive_atoms.set_positions(posp)
    
    a,_,c,_,_,_ = primitive_atoms.get_cell_lengths_and_angles()
    # only a is used by QE
    b,alpha,beta,gamma = [0.]*4
    cell_par = [a,b,c,alpha,beta,gamma]
    return primitive_atoms,cell_par,inequivalent_pos

### check all

In [153]:
sg = 167
cc = crystals[sg][0]
primitive_atoms,cell_par,inequivalent_pos = get_ibrav5_atoms(cc)
view(cc)
view(primitive_atoms)
print 'sg {}'.format(sg)
print cell_par


sg 167
[9.7049669483204024, 0.0, 0.0, 119.39306125170683, 0.0, 0.0]


In [154]:
symprec = 1e-5
print spg.get_spacegroup(cc,symprec=symprec)
print spg.get_spacegroup(primitive_atoms,symprec=symprec)

R-3c (167)
R-3c (167)


In [147]:
for sg in relevant_sg:
    try:
        cc = crystals[sg][0]
        primitive_atoms,cell_par,inequivalent_pos = get_ibrav5_atoms(cc)
        view(cc)
        view(primitive_atoms)
        print 'sg {}'.format(sg)
        print cell_par
        raw_input("Press Enter to continue...")
    except:
        print 'No structure for sg {}'.format(sg)



sg 146
[3.7820019948990224, 0.0, 0.0, 0.0, 0.0, 0.0]
Press Enter to continue...
sg 148
[5.4039213269642969, 0.0, 0.0, 0.0, 0.0, 0.0]
Press Enter to continue...
sg 155
[4.4237159514123023, 0.0, 0.0, 0.0, 0.0, 0.0]
No structure for sg 155
sg 160
[5.9663080782218865, 0.0, 0.0, 0.0, 0.0, 0.0]
No structure for sg 160
sg 161
[5.4425698931175646, 0.0, 0.0, 0.0, 0.0, 0.0]
No structure for sg 161
sg 166
[10.209972340390793, 0.0, 0.0, 0.0, 0.0, 0.0]
No structure for sg 166
sg 167
[9.7049669483204024, 0.0, 0.0, 0.0, 0.0, 0.0]
No structure for sg 167


## ibrav 8
8          Orthorhombic P                  celldm(2)=b/a
                                             celldm(3)=c/a
                                             
      v1 = (a,0,0),  v2 = (0,b,0), v3 = (0,0,c)
      
PROBLEM -> refine cell

In [12]:
a = 2.21587422;b =  8.48205277; c = 3.8134843
v1 = a*np.array([1,0,0]);   v2 = np.array([0,b,0]);  
v3 = np.array([0,0,c])
print np.array([v1,v2,v3])

[[ 2.21587422  0.          0.        ]
 [ 0.          8.48205277  0.        ]
 [ 0.          0.          3.8134843 ]]


In [120]:
relevant_sg = []
for key,item in sg2ibrav.iteritems():
    if item == 8:
        relevant_sg.append(key)
print relevant_sg

[16, 17, 18, 19, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62]


In [118]:
sg = 47
cc = crystals[sg][3]
print cc.get_cell()
print cc.get_cell_lengths_and_angles()
Na = cc.get_number_of_atoms()
pos = cc.get_scaled_positions()[0]
cell_par = cc.get_cell_lengths_and_angles()
view(cc)

[[  4.52402677e+00  -1.23243555e-09   1.33360400e-09]
 [  1.23565798e-09   3.86522577e+00  -6.19755176e-01]
 [ -1.32583218e-09  -6.99007899e-01   3.85168169e+00]]
[   4.52402677    3.91459662    3.91459626  109.39549747   90.           90.        ]


In [126]:
sg = 47
cc = crystals[sg][3]
symprec = get_symprec(cc, sg)
# (lattice, positions, numbers) = spg.find_primitive(cc, symprec=1e-05, angle_tolerance=-1.0)
(lattice, positions, numbers) = spg.refine_cell(cc,symprec=1e-05)
# (lattice, positions, numbers) = spg.standardize_cell(
#                             cc, to_primitive=False,no_idealize=False,
#                             symprec=symprec, angle_tolerance=-1.0)
ee = ase.Atoms(cell=lattice,scaled_positions=positions,numbers=numbers)
cell = ee.get_cell()
pos = ee.get_positions()

print cell
print ee.get_cell_lengths_and_angles()

[[ 2.26220493  0.          0.        ]
 [ 0.          3.19476042  0.        ]
 [ 0.          0.          4.52402677]]
[  2.26220493   3.19476042   4.52402677  90.          90.          90.        ]


## ibrav 9
9          Orthorhombic base-centered(bco) celldm(2)=b/a
                                             celldm(3)=c/a
                                             
      v1 = (a/2, b/2,0),  v2 = (-a/2,b/2,0),  v3 = (0,0,c)
      
-9          as 9, alternate description

      v1 = (a/2,-b/2,0),  v2 = (a/2, b/2,0),  v3 = (0,0,c)
      
USE IBRAV=-9 AND PRIMITIVE CELL TRANSFORMATION   

UPDATE: IBRAV=-9 is not compatible with spacegroup input -> IBRAV0

In [104]:
a = 3.20677445;b =  7.72912965; c = 6.17850825 
# v1 = 0.5*np.array([a,-b,0]);   
# v2 = 0.5*np.array([a,b,0]);  
# v3 = np.array([0,0,c])
v1 = 0.5*np.array([a,b,0]);   
v2 = 0.5*np.array([-a,b,0]);  
v3 = np.array([0,0,c])
print np.array([v1,v2,v3])
print get_relative_angle(v1,v2,radian=False),get_relative_angle(v1,v3,radian=False),get_relative_angle(v3,v2,radian=False)

[[ 1.60338723  3.86456482  0.        ]
 [-1.60338723  3.86456482  0.        ]
 [ 0.          0.          6.17850825]]
45.0665999947 90.0 90.0


In [11]:
relevant_sg = []
for key,item in sg2ibrav.iteritems():
    if item == 9:
        relevant_sg.append(key)
print relevant_sg

[20, 21, 35, 36, 37, 38, 39, 40, 41, 63, 64, 65, 66, 67, 68]


### investigate the prob

In [95]:
sg = 68
cc = crystals[sg][3]
print cc.get_cell()
print cc.get_cell_lengths_and_angles()
Na = cc.get_number_of_atoms()
pos = cc.get_scaled_positions()[0]
cell_par = cc.get_cell_lengths_and_angles()
view(cc)

[[  3.20677445e+00   1.43959911e-14   1.65889644e-15]
 [  2.00083219e-14   7.72912965e+00  -2.10309227e-16]
 [ -4.30211684e-15  -2.13417366e-16   6.17850825e+00]]
[  3.20677445   7.72912965   6.17850825  90.          90.          90.        ]


In [111]:
sg = 68
cc = crystals[sg][3]
symprec = get_symprec(cc, sg)
# (lattice, positions, numbers) = spg.find_primitive(cc, symprec=1e-05, angle_tolerance=-1.0)
(lattice, positions, numbers) = spg.standardize_cell(
                            cc, to_primitive=True,no_idealize=False,
                            symprec=symprec, angle_tolerance=-1.0)
ee = ase.Atoms(cell=lattice,scaled_positions=positions,numbers=numbers)
cell = ee.get_cell()
pos = ee.get_positions()

print cell
print ee.get_cell_lengths_and_angles()

[[ 1.60338723 -3.86456482  0.        ]
 [ 1.60338723  3.86456482  0.        ]
 [ 0.          0.          6.17850825]]
[   4.18398278    4.18398278    6.17850825   90.           90.
  134.93339995]


In [114]:
dd = unskewCell(ee)
print dd.get_cell()
print dd.get_cell_lengths_and_angles()

[[ 1.60338723 -3.86456482  0.        ]
 [ 3.20677445  0.          0.        ]
 [ 0.          0.          6.17850825]]
[  4.18398278   3.20677445   6.17850825  90.          90.          67.46669997]


### functions to solve it

In [86]:
def get_ibrav9_frame(frame, sg):
    symprec = get_symprec(frame, sg)
    if symprec is None:
        print 'Not possible'
        return None
    (lattice, positions, numbers) = spg.standardize_cell(
                            frame, to_primitive=True,no_idealize=False,
                            symprec=symprec, angle_tolerance=-1.0)
    primitive_atoms = ase.Atoms(cell=lattice, scaled_positions=positions, numbers=numbers)

    pos = primitive_atoms.get_positions()

    inequivalent_pos = pos[0]

    a, b, c, _, _, _ = primitive_atoms.get_cell_lengths_and_angles()
    # only a is used by QE
    alpha, beta, gamma = [0.] * 3
    cell_par = [a, b, c, alpha, beta, gamma]
    return primitive_atoms, cell_par, inequivalent_pos

### check all

In [87]:
int(spg.get_spacegroup(cc,symprec=1e-3).split(' ')[1][1:-1])

69

In [93]:
sg = 68
cc = crystals[sg][3]
primitive_atoms,cell_par,inequivalent_pos = get_ibrav9_frame(cc,sg)
# view(cc)
view(primitive_atoms)
print 'sg {}'.format(sg)
print primitive_atoms.get_cell_lengths_and_angles()


sg 68
[   4.18398278    4.18398278    6.17850825   90.           90.
  134.93339995]


In [91]:
symprec = 1e-3
print spg.get_spacegroup(cc,symprec=symprec)
print spg.get_spacegroup(primitive_atoms,symprec=symprec)

Fmmm (69)
Fmmm (69)


In [19]:
for sg in relevant_sg:
    try:
        cc = crystals[sg][3]
        
        primitive_atoms,cell_par,inequivalent_pos = get_ibrav9_atoms(cc,sg)
        view(cc)
        view(primitive_atoms)
        print 'sg {}'.format(sg)
        print cell_par
        raw_input("Press Enter to continue...")
    except:
        print 'No structure for sg {}'.format(sg)



sg 20
[2.6649982073776921, 2.6649982073776921, 4.7160166958629146, 0.0, 0.0, 0.0]
Press Enter to continue...
sg 21
[4.1690069584293505, 4.1690069584293505, 3.1600226818471424, 0.0, 0.0, 0.0]
Press Enter to continue...
sg 35
[2.2474941866587579, 2.2474941866587579, 2.1998670905715314, 0.0, 0.0, 0.0]
Press Enter to continue...
sg 36
[3.1447992005676988, 3.1447992005676988, 3.8912346352252323, 0.0, 0.0, 0.0]
Press Enter to continue...
sg 37
[4.5418060211975213, 4.5418060211975213, 2.2821125827075366, 0.0, 0.0, 0.0]
Press Enter to continue...
sg 38
[3.1338742732981419, 3.1338742732981419, 4.432479786791002, 0.0, 0.0, 0.0]
Press Enter to continue...
sg 39
[4.2704818849539832, 4.2704818849539832, 3.8463682540392385, 0.0, 0.0, 0.0]
Press Enter to continue...
sg 40
[2.2523758067186388, 2.2523758067186388, 8.1096037255931535, 0.0, 0.0, 0.0]
Press Enter to continue...
sg 41
[2.2401064064736516, 2.2401064064736516, 7.5060083929890498, 0.0, 0.0, 0.0]
Press Enter to continue...
sg 63
[2.26120700728

## ibrav 10
10          Orthorhombic face-centered      celldm(2)=b/a
                                             celldm(3)=c/a
                                             
      v1 = (a/2,0,c/2),  v2 = (a/2,b/2,0),  v3 = (0,b/2,c/2)

NOT STANDARD PRIMITIVE CELL 

In [152]:
a =14.373234;b =  3.25260783; c =  3.05625406 
v1 = 0.5*np.array([a,0,c]);   
v2 = 0.5*np.array([a,b,0]);  
v3 = 0.5*np.array([0,b,c])
print np.array([v1,v2,v3])
print np.linalg.norm(np.array([v1,v2,v3]),axis=1)
print get_relative_angle(v1,v2,radian=False),get_relative_angle(v1,v3,radian=False),get_relative_angle(v3,v2,radian=False)

[[ 7.186617    0.          1.52812703]
 [ 7.186617    1.62630392  0.        ]
 [ 0.          1.62630392  1.52812703]]
[ 7.34728767  7.36833281  2.23159957]
17.4441621261 81.8120067773 80.7438310966


In [21]:
relevant_sg = []
for key,item in sg2ibrav.iteritems():
    if item == 10:
        relevant_sg.append(key)
print relevant_sg

[22, 42, 43, 69, 70]


### investigate the prob

In [151]:
sg = 42
cc = crystals[sg][4]
print cc.get_cell()
print cc.get_cell_lengths_and_angles()
Na = cc.get_number_of_atoms()
pos = cc.get_scaled_positions()[0]
cell_par = cc.get_cell_lengths_and_angles()
view(cc)

[[  8.86468550e+00  -1.46784157e-14   7.50578510e-14]
 [ -1.01775700e-14   4.43278654e+00   5.12818624e-17]
 [  4.03327015e-14   6.09307556e-17   4.43190334e+00]]
[  8.8646855    4.43278654   4.43190334  90.          90.          90.        ]


In [153]:
sg = 42
cc = crystals[sg][4]
(lattice, positions, numbers) = spg.find_primitive(cc, symprec=1e-05, angle_tolerance=-1.0)
ee = ase.Atoms(cell=lattice,scaled_positions=positions,numbers=numbers)
cell = ee.get_cell()
pos = ee.get_positions()
v2 = cell[0]
axis = [0,0,1]
theta = - 60*np.pi/180.
rot = rotation_matrix(axis, theta)
v2p = np.dot(rot,v2)
cellp = np.dot(cell,rot.T)
posp = np.dot(pos,rot.T)
print cell
print cellp
print ee.get_cell_lengths_and_angles()
ff = unskewCell(ee)
print ff.get_cell_lengths_and_angles()
view(ff)

[[ 0.          2.21639327  4.43234275]
 [ 2.21595167  0.          4.43234275]
 [ 2.21595167  2.21639327  0.        ]]
[[ 1.91945288  1.10819664  4.43234275]
 [ 1.10797584 -1.91907044  4.43234275]
 [ 3.02742871 -0.81087381  0.        ]]
[  4.95560909   4.9554116    3.13414118  71.56846865  71.56161851
  36.86991284]
[   3.13414118    4.9554116     3.13414118   71.56846865   89.9885831
  108.43153135]


### functions to solve it

In [98]:
def get_ibrav0_atoms(frame,sg):
    import spglib as spg
    symprec = get_symprec(frame,sg)
    if symprec is None:
        print 'Not possible'
        return None
    (lattice, positions, numbers) = spg.find_primitive(frame, symprec=symprec, angle_tolerance=-1.0)
    primitive_atoms = ase.Atoms(cell=lattice,scaled_positions=positions,numbers=numbers)
    
    cell = primitive_atoms.get_cell()
    pos = primitive_atoms.get_positions()
    
    inequivalent_pos = pos[0]
    
    cell_par = primitive_atoms.get_cell_lengths_and_angles()
    return primitive_atoms,cell_par,inequivalent_pos

## ibrav 11
11          Orthorhombic body-centered      celldm(2)=b/a
                                             celldm(3)=c/a
                                             
      v1=(a/2,b/2,c/2),  v2=(-a/2,b/2,c/2),  v3=(-a/2,-b/2,c/2)
      
      NOT STANDARD PRIMITIVE CELL 

In [150]:
a,b,c = [2.2564751 ,   7.53571616 ,  7.18915049]
v1 = 0.5*np.array([a,b,c]);   
v2 = 0.5*np.array([-a,b,c]);  
v3 = 0.5*np.array([-a,-b,c])
print np.array([v1,v2,v3])
print np.linalg.norm(np.array([v1,v2,v3]),axis=1)
print get_relative_angle(v1,v2,radian=False),get_relative_angle(v1,v3,radian=False),get_relative_angle(v3,v2,radian=False)

[[ 1.12823755  3.76785808  3.59457525]
 [-1.12823755  3.76785808  3.59457525]
 [-1.12823755 -3.76785808  3.59457525]]
[ 5.32828731  5.32828731  5.32828731]
24.4492524307 95.1505275281 90.0057785886


In [146]:
relevant_sg = []
for key,item in sg2ibrav.iteritems():
    if item == 11:
        relevant_sg.append(key)
print relevant_sg

[23, 24, 44, 45, 46, 71, 72, 73, 74]


### investigate the prob

In [147]:
sg = 24
cc = crystals[sg][4]
print cc.get_cell()
print cc.get_cell_lengths_and_angles()
Na = cc.get_number_of_atoms()
pos = cc.get_scaled_positions()[0]
cell_par = cc.get_cell_lengths_and_angles()
view(cc)

[[  2.25647510e+00  -1.07309529e-14   5.90204215e-18]
 [ -3.10996574e-14   7.53571616e+00  -4.41210014e-13]
 [  3.41551919e-17  -4.62107828e-13   7.18915049e+00]]
[  2.2564751    7.53571616   7.18915049  90.          90.          90.        ]


In [149]:
sg = 24
cc = crystals[sg][4]
(lattice, positions, numbers) = spg.find_primitive(cc, symprec=1e-05, angle_tolerance=-1.0)
ee = ase.Atoms(cell=lattice,scaled_positions=positions,numbers=numbers)
cell = ee.get_cell()
pos = ee.get_positions()
v2 = cell[0]
axis = [0,0,1]
theta = - 60*np.pi/180.
rot = rotation_matrix(axis, theta)
v2p = np.dot(rot,v2)
cellp = np.dot(cell,rot.T)
posp = np.dot(pos,rot.T)
print cell
print cellp
print ee.get_cell_lengths_and_angles()
ff = unskewCell(ee)
print ff.get_cell_lengths_and_angles()
view(ff)

[[-1.12823755  3.59457525  3.76785808]
 [ 1.12823755 -3.59457525  3.76785808]
 [ 1.12823755  3.59457525 -3.76785808]]
[[ 2.5488747   2.77437     3.76785808]
 [-2.5488747  -2.77437     3.76785808]
 [ 3.67711225  0.82020524 -3.76785808]]
[   5.32828731    5.32828731    5.32828731  155.55074756   95.15052749
   89.99422145]
[   5.32828731    5.32828731    2.2564751    77.77537378  102.22462622
   89.99422145]


In [142]:
cellp[np.abs(cellp)<1e-10] = 0.
ee.set_cell(cellp)
ee.set_positions(posp)
a = ee.get_cell_lengths_and_angles()[0]
c = np.cos(ee.get_cell_lengths_and_angles()[3]*np.pi/180.)
view(ee)
print ee.get_cell()
print ee.get_cell_lengths_and_angles()
print a,c

[[ 3.60572805 -2.08176806  1.49473402]
 [ 0.          4.16353612  1.49473402]
 [-3.60572805 -2.08176806  1.49473402]]
[   4.42371595    4.42371595    4.42371595  109.192587    109.192587
  109.192587  ]
4.42371595141 -0.328744459329


### functions to solve it

In [87]:
def get_ibrav11_atoms(frame,sg):
    import spglib as spg
    symprec = get_symprec(frame,sg)
    if symprec is None:
        print 'Not possible'
        return None
    (lattice, positions, numbers) = spg.find_primitive(frame, symprec=symprec, angle_tolerance=-1.0)
    primitive_atoms = ase.Atoms(cell=lattice,scaled_positions=positions,numbers=numbers)

    cell = primitive_atoms.get_cell()
    pos = primitive_atoms.get_positions()
    
    inequivalent_pos = pos[0]
    
    a,b,c,alpha,beta,gamma = primitive_atoms.get_cell_lengths_and_angles()
    # only a is used by QE
    
    cell_par = [a,b,c,alpha,beta,gamma]
    return primitive_atoms,cell_par,inequivalent_pos

## ibrav 12
12          Monoclinic P, unique axis c     celldm(2)=b/a
                                             celldm(3)=c/a,
                                             celldm(4)=cos(ab)
                                             
      v1=(a,0,0), v2=(b*cos(gamma),b*sin(gamma),0),  v3 = (0,0,c)
      
      where gamma is the angle between axis a and b.
      
-12          Monoclinic P, unique axis b     celldm(2)=b/a
                                             celldm(3)=c/a,
                                             celldm(5)=cos(ac)
                                             
      v1 = (a,0,0), v2 = (0,b,0), v3 = (c*cos(beta),0,c*sin(beta))
      
      where beta is the angle between axis a and c

go to rotation of the cell and -12

In [67]:
d2r = np.pi / 180.
a,b,c,alpha,beta,gamma = [  3.89150141 ,  4.39989082 ,  4.49522541 , 90.      ,    29.99855789 , 90.       ]
# v1 = np.array([a,0,0]);   
# v2 = np.array([b*np.cos(beta*d2r),b*np.sin(beta*d2r),0])
# v3 = np.array([0,0,c]);  
v1 = np.array([a,0,0]);   
v2 = np.array([0,b,0])
v3 = np.array([c*np.cos(beta*d2r),0,c*np.sin(beta*d2r)]);  
print np.array([v1,v2,v3])
print get_relative_angle(v1,v2,radian=False),get_relative_angle(v1,v3,radian=False),get_relative_angle(v3,v2,radian=False)

[[ 3.89150141  0.          0.        ]
 [ 0.          4.39989082  0.        ]
 [ 3.89303597  0.          2.24751472]]
90.0 29.99855789 90.0


In [72]:
relevant_sg = []
for key,item in sg2ibrav.iteritems():
    if item == -12:
        relevant_sg.append(key)
print relevant_sg

[3, 4, 6, 7, 10, 11, 13, 14]


### investigate the prob

In [174]:
(lattice, positions, numbers) = spg.refine_cell(cc,symprec=1e-05)
v1,v2,v3 = lattice
print lattice
print get_relative_angle(v1,v2,radian=False),get_relative_angle(v1,v3,radian=False),get_relative_angle(v3,v2,radian=False)
primitive_atoms = ase.Atoms(cell=lattice,scaled_positions=positions,numbers=numbers)
view(primitive_atoms)
view(cc)

[[  2.24751524e+00   0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   4.39989082e+00   0.00000000e+00]
 [ -2.65704282e-03   0.00000000e+00   3.89150050e+00]]
90.0 90.0391204667 90.0


In [169]:
sg = 10
cc =  crystals[sg][0]
# cc = unskewCell( crystals[sg][0])
cell = cc.get_cell()
cell[np.abs(cell)<1e-10] = 0.
print cell
print cc.get_cell_lengths_and_angles()
Na = cc.get_number_of_atoms()
pos = cc.get_scaled_positions()[0]
cell_par = cc.get_cell_lengths_and_angles()
#view(cc)
axis = [0,1,0]
theta = get_relative_angle(cell[0],[1,0,0],radian=True)
rot = rotation_matrix(axis, theta)
cellp = np.dot(cell,rot.T)
cellp[np.abs(cellp)<1e-10] = 0.
print cellp

[[ 3.8692221   0.          0.41581676]
 [ 0.          4.39989082  0.        ]
 [ 3.63059525  0.          2.65062816]]
[  3.89150141   4.39989082   4.49522541  90.          29.99855789  90.        ]
[[ 3.89150141  0.          0.        ]
 [ 0.          4.39989082  0.        ]
 [ 3.89303597  0.          2.24751472]]


In [137]:
get_relative_angle(cell[0],[1,0,0],radian=False)

17.115409396517464

### functions to solve it

In [73]:
def get_ibrav12_frame(frame, sg):
    symprec = get_symprec(frame, sg)
    if symprec is None:
        print 'Not possible'
        return None
    (lattice, positions, numbers) = spg.standardize_cell(
                            frame, to_primitive=False,no_idealize=False,
                            symprec=symprec, angle_tolerance=-1.0)
    primitive_atoms = ase.Atoms(cell=lattice, scaled_positions=positions, numbers=numbers)


    pos = primitive_atoms.get_positions()

    inequivalent_pos = pos[0]

    cell_par = primitive_atoms.get_cell_lengths_and_angles()
    return primitive_atoms, cell_par, inequivalent_pos

### check all

In [74]:
int(spg.get_spacegroup(cc,symprec=1e-3).split(' ')[1][1:-1])

69

In [76]:
sg = 10
cc = crystals[sg][0]
primitive_atoms,cell_par,inequivalent_pos = get_ibrav12_frame(cc,sg)
view(cc)
view(primitive_atoms)
print 'sg {}'.format(sg)
print cell_par


sg 10
[  2.24751524   4.39989082   3.89150141  90.          90.03912047  90.        ]


In [77]:
symprec = 1e-3
print spg.get_spacegroup(cc,symprec=symprec)
print spg.get_spacegroup(primitive_atoms,symprec=symprec)

P2/m (10)
P2/m (10)


In [79]:
for sg in relevant_sg:
    try:
        cc = crystals[sg][0]
        
        primitive_atoms,cell_par,inequivalent_pos = get_ibrav12_frame(cc,sg)
        view(cc)
        view(primitive_atoms)
        print 'sg {}'.format(sg)
        print cell_par
        print primitive_atoms.get_cell()
        raw_input("Press Enter to continue...")
    except:
        print 'No structure for sg {}'.format(sg)


sg 3
[  2.24563515   2.19916736   3.8925369   90.          90.01838927  90.        ]
[[  2.24563515e+00   0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   2.19916736e+00   0.00000000e+00]
 [ -1.24932240e-03   0.00000000e+00   3.89253670e+00]]
Press Enter to continue...
sg 4
[  2.26207006   2.26079637   3.19911424  90.          90.02059709  90.        ]
[[  2.26207006e+00   0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   2.26079637e+00   0.00000000e+00]
 [ -1.15004001e-03   0.00000000e+00   3.19911403e+00]]
Press Enter to continue...
sg 6
[   2.24605238    4.4025935     2.24682181   90.          119.97661982
   90.        ]
[[ 2.24605238  0.          0.        ]
 [ 0.          4.4025935   0.        ]
 [-1.12261681  0.          1.94626303]]
Press Enter to continue...
sg 7
[   2.25963971    2.25914046    3.91576514   90.          125.19854566
   90.        ]
[[ 2.25963971  0.          0.        ]
 [ 0.          2.25914046  0.        ]
 [-2.25709235  0.          3.1998048 ]]
Pre

## ibrav 13
13          Monoclinic base-centered        celldm(2)=b/a
                                             celldm(3)=c/a,
                                             celldm(4)=cos(ab)
                                             
      v1 = (  a/2,         0,                -c/2),
      
      v2 = (b*cos(gamma), b*sin(gamma), 0),
      
      v3 = (  a/2,         0,                  c/2),
      
      where gamma is the angle between axis a and b

        ibrav0

[len(a), len(b), len(c), angle(a,b), angle(a,c), angle(b,c)]

In [81]:
d2r = np.pi / 180.
a,b,c,alpha,beta,gamma = [3.91330814 ,  4.52602357  , 3.19325456 , 90.   ,       35.31423407 , 90.    ]
v1 = np.array([a/2.,0,-c/2.]);   
v2 = np.array([b*np.cos(beta*d2r),b*np.sin(beta*d2r),0])
v3 = np.array([a/2.,0,c/2.]);  
print np.array([v1,v2,v3])
print get_relative_angle(v1,v2,radian=False),get_relative_angle(v1,v3,radian=False),get_relative_angle(v3,v2,radian=False)

[[ 1.95665407  0.         -1.59662728]
 [ 3.69320811  2.61631482  0.        ]
 [ 1.95665407  0.          1.59662728]]
50.7858696701 78.4288044502 50.7858696701


In [82]:
relevant_sg = []
for key,item in sg2ibrav.iteritems():
    if item == 13:
        relevant_sg.append(key)
print relevant_sg

[5, 8, 9, 12, 15]


### investigate the prob

In [187]:
sg = 8
cc = crystals[sg][0]
print cc.get_cell()
print cc.get_cell_lengths_and_angles()
Na = cc.get_number_of_atoms()
pos = cc.get_scaled_positions()[0]
cell_par = cc.get_cell_lengths_and_angles()
view(cc)

[[  3.89221850e+00   1.46737067e-16   4.05728750e-01]
 [  3.59362983e-17   4.52602357e+00  -2.78845593e-17]
 [  2.40025319e+00   6.44164412e-17   2.10610049e+00]]
[  3.91330814   4.52602357   3.19325456  90.          35.31423407  90.        ]


In [185]:
spg.get_spacegroup(cc,symprec=1e-5)

u'C2/m (12)'

In [186]:
ff = unskewCell(cc)
view(ff)

In [180]:
sg = 8
cc = crystals[sg][0]
(lattice, positions, numbers) = spg.refine_cell(cc,symprec=1e-05)
v1,v2,v3 = lattice
print lattice
print get_relative_angle(v1,v2,radian=False),get_relative_angle(v1,v3,radian=False),get_relative_angle(v3,v2,radian=False)
primitive_atoms = ase.Atoms(cell=lattice,scaled_positions=positions,numbers=numbers)
view(primitive_atoms)
view(cc)

[[ 3.91330814  0.          0.        ]
 [ 0.          2.26301179  0.        ]
 [-1.30763156  0.          1.84589388]]
90.0 125.31376392 90.0


In [113]:
sg = 8
cc = crystals[sg][0]
(lattice, positions, numbers) = spg.find_primitive(cc, symprec=1e-05, angle_tolerance=-1.0)
ee = ase.Atoms(cell=lattice,scaled_positions=positions,numbers=numbers)
print ee.get_cell_lengths_and_angles()
cell = ee.get_cell()
pos = ee.get_positions()
v2 = cell[0]
axis = [0,0,1]
theta = - 60*np.pi/180.
rot = rotation_matrix(axis, theta)
v2p = np.dot(rot,v2)
cellp = np.dot(cell,rot.T)
posp = np.dot(pos,rot.T)
print cell
print cellp

[   2.26026564    2.26026564    2.26212832  120.02688112  120.02688112
   60.08039791]
[[ 1.95665407 -1.13150589  0.        ]
 [ 1.95665407  1.13150589  0.        ]
 [-1.30763156  0.          1.84589388]]
[[ -1.58581152e-03  -2.26026508e+00   0.00000000e+00]
 [  1.95823988e+00  -1.12875919e+00   0.00000000e+00]
 [ -6.53815782e-01   1.13244215e+00   1.84589388e+00]]


In [139]:
a = 4.42371595; c = np.cos(109.192587*np.pi/180.)
tx = np.sqrt((1-c)/2); ty = np.sqrt((1-c)/6); tz = np.sqrt((1+2*c)/3)
v1 = a*np.array([tx,-ty,tz]);   v2 = a*np.array([0,2*ty,tz]);   v3 = a*np.array([-tx,-ty,tz])
print np.array([v1,v2,v3])

[[ 3.60572805 -2.08176806  1.49473402]
 [ 0.          4.16353612  1.49473402]
 [-3.60572805 -2.08176806  1.49473402]]


In [142]:
cellp[np.abs(cellp)<1e-10] = 0.
ee.set_cell(cellp)
ee.set_positions(posp)
a = ee.get_cell_lengths_and_angles()[0]
c = np.cos(ee.get_cell_lengths_and_angles()[3]*np.pi/180.)
view(ee)
print ee.get_cell()
print ee.get_cell_lengths_and_angles()
print a,c

[[ 3.60572805 -2.08176806  1.49473402]
 [ 0.          4.16353612  1.49473402]
 [-3.60572805 -2.08176806  1.49473402]]
[   4.42371595    4.42371595    4.42371595  109.192587    109.192587
  109.192587  ]
4.42371595141 -0.328744459329


In [83]:
def get_ibrav0_frame(frame, sg):
    symprec = get_symprec(frame, sg)
    if symprec is None:
        print 'Not possible'
        return None
    (lattice, positions, numbers) = spg.standardize_cell(
                            frame, to_primitive=True,no_idealize=False,
                            symprec=symprec, angle_tolerance=-1.0)
    primitive_atoms = ase.Atoms(cell=lattice, scaled_positions=positions, numbers=numbers)

    cell = primitive_atoms.get_cell()
    pos = primitive_atoms.get_positions()

    inequivalent_pos = pos[0]

    return primitive_atoms, cell, inequivalent_pos

### check all

In [84]:
for sg in relevant_sg:
    try:
        cc = crystals[sg][0]
        
        primitive_atoms,cell_par,inequivalent_pos = get_ibrav0_frame(cc,sg)
        view(cc)
        view(primitive_atoms)
        print 'sg {}'.format(sg)
        print cell_par
        print primitive_atoms.get_cell()
        raw_input("Press Enter to continue...")
    except:
        print 'No structure for sg {}'.format(sg)


sg 5
[[  1.95804449e+00  -1.12994073e+00   0.00000000e+00]
 [  1.95804449e+00   1.12994073e+00   0.00000000e+00]
 [ -7.22283768e-04   0.00000000e+00   3.69303824e+00]]
[[  1.95804449e+00  -1.12994073e+00   0.00000000e+00]
 [  1.95804449e+00   1.12994073e+00   0.00000000e+00]
 [ -7.22283768e-04   0.00000000e+00   3.69303824e+00]]
Press Enter to continue...
sg 8
[[ 1.95665407 -1.13150589  0.        ]
 [ 1.95665407  1.13150589  0.        ]
 [-1.30763156  0.          1.84589388]]
[[ 1.95665407 -1.13150589  0.        ]
 [ 1.95665407  1.13150589  0.        ]
 [-1.30763156  0.          1.84589388]]
Press Enter to continue...
sg 9
[[ 1.95910702 -1.13055256  0.        ]
 [ 1.95910702  1.13055256  0.        ]
 [ 1.31166833  0.          3.6905804 ]]
[[ 1.95910702 -1.13055256  0.        ]
 [ 1.95910702  1.13055256  0.        ]
 [ 1.31166833  0.          3.6905804 ]]
Press Enter to continue...
sg 12
[[ 1.95898807 -1.13027104  0.        ]
 [ 1.95898807  1.13027104  0.        ]
 [-1.31020459  0.     

## ibrav 14
14          Triclinic                       celldm(2)= b/a,
                                             celldm(3)= c/a,
                                             celldm(4)= cos(bc),
                                             celldm(5)= cos(ac),
                                             celldm(6)= cos(ab)
                                             
      v1 = (a, 0, 0),
      
      v2 = (b*cos(gamma), b*sin(gamma), 0)
      
      v3 = (c*cos(beta),  c*(cos(alpha)-cos(beta)cos(gamma))/sin(gamma),
           c*sqrt( 1 + 2*cos(alpha)cos(beta)cos(gamma)
                     - cos(alpha)^2-cos(beta)^2-cos(gamma)^2 )/sin(gamma) )
                     
      where alpha is the angle between axis b and c
             beta is the angle between axis a and c
            gamma is the angle between axis a and b

In [ ]:
d2r = np.pi / 180.
a,b,c,alpha,beta,gamma = [3.91330814 ,  4.52602357  , 3.19325456 , 90.   ,       35.31423407 , 90.    ]
v1 = np.array([a/2.,0,0]);   
v2 = np.array([b*np.cos(gamma*d2r),b*np.sin(gamma*d2r),0])
v3 = np.array([c*np.cos(beta),c*(np.cos(alpha)-np.cos(beta)*np.cos(gamma))/np.sin(gamma),
              c*np.sqrt( 1 + 2*np.cos(alpha)*np.cos(beta)*np.cos(gamma)
                 - np.cos(alpha)**2-np.cos(beta)**2-np.cos(gamma)**2 )/np.sin(gamma)
              ]);  
print np.array([v1,v2,v3])
print get_relative_angle(v1,v2,radian=False),get_relative_angle(v1,v3,radian=False),get_relative_angle(v3,v2,radian=False)

In [190]:
relevant_sg = []
for key,item in sg2ibrav.iteritems():
    if item == 14:
        relevant_sg.append(key)
print relevant_sg

[1, 2]


### investigate the prob

In [192]:
sg = 15
cc = crystals[sg][0]
print cc.get_cell()
print cc.get_cell_lengths_and_angles()
Na = cc.get_number_of_atoms()
pos = cc.get_scaled_positions()[0]
cell_par = cc.get_cell_lengths_and_angles()
view(cc)

[[  1.14454199e+01  -8.60591502e-13   1.00921837e+00]
 [ -1.38158488e-13   2.21979427e+00   3.07769495e-13]
 [  3.26041037e+00   2.11191674e-13   3.09863867e+00]]
[ 11.48982847   2.21979427   4.49798149  90.          38.50361739  90.        ]


In [111]:
sg = 155
cc = crystals[sg][0]
(lattice, positions, numbers) = spg.find_primitive(cc, symprec=1e-05, angle_tolerance=-1.0)
ee = ase.Atoms(cell=lattice,scaled_positions=positions,numbers=numbers)
cell = ee.get_cell()
pos = ee.get_positions()
v2 = cell[0]
axis = [0,0,1]
theta = - 60*np.pi/180.
rot = rotation_matrix(axis, theta)
v2p = np.dot(rot,v2)
cellp = np.dot(cell,rot.T)
posp = np.dot(pos,rot.T)
print cell
print cellp

[[ 3.60572805  2.08176806  1.49473402]
 [-3.60572805  2.08176806  1.49473402]
 [ 0.         -4.16353612  1.49473402]]
[ 3.60572805 -2.08176806  1.49473402]
[[  3.60572805e+00  -2.08176806e+00   1.49473402e+00]
 [ -1.11022302e-15   4.16353612e+00   1.49473402e+00]
 [ -3.60572805e+00  -2.08176806e+00   1.49473402e+00]]


In [139]:
a = 4.42371595; c = np.cos(109.192587*np.pi/180.)
tx = np.sqrt((1-c)/2); ty = np.sqrt((1-c)/6); tz = np.sqrt((1+2*c)/3)
v1 = a*np.array([tx,-ty,tz]);   v2 = a*np.array([0,2*ty,tz]);   v3 = a*np.array([-tx,-ty,tz])
print np.array([v1,v2,v3])

[[ 3.60572805 -2.08176806  1.49473402]
 [ 0.          4.16353612  1.49473402]
 [-3.60572805 -2.08176806  1.49473402]]


In [142]:
cellp[np.abs(cellp)<1e-10] = 0.
ee.set_cell(cellp)
ee.set_positions(posp)
a = ee.get_cell_lengths_and_angles()[0]
c = np.cos(ee.get_cell_lengths_and_angles()[3]*np.pi/180.)
view(ee)
print ee.get_cell()
print ee.get_cell_lengths_and_angles()
print a,c

[[ 3.60572805 -2.08176806  1.49473402]
 [ 0.          4.16353612  1.49473402]
 [-3.60572805 -2.08176806  1.49473402]]
[   4.42371595    4.42371595    4.42371595  109.192587    109.192587
  109.192587  ]
4.42371595141 -0.328744459329


### functions to solve it

In [67]:
def get_ibrav5_atoms(frame,sg):
    import spglib as spg
    symprec = get_symprec(frame,sg)
    if symprec is None:
        print 'Not possible'
        return None
    (lattice, positions, numbers) = spg.find_primitive(frame, symprec=symprec, angle_tolerance=-1.0)
    primitive_atoms = ase.Atoms(cell=lattice,scaled_positions=positions,numbers=numbers)

    cell = primitive_atoms.get_cell()
    pos = primitive_atoms.get_positions()
    
    axis = [0,0,1]
    theta = - 60.*np.pi/180.
    rot = rotation_matrix(axis, theta)
    
    cellp = np.dot(cell,rot.T)
    cellp[np.abs(cellp)<1e-10] = 0.
    posp = np.dot(pos,rot.T)
    inequivalent_pos = posp[0]
    
    primitive_atoms.set_cell(cellp)
    primitive_atoms.set_positions(posp)
    
    a,_,_,alpha,_,_ = primitive_atoms.get_cell_lengths_and_angles()
    # only a is used by QE
    b,c,beta,gamma = [0.]*4
    cell_par = [a,b,c,alpha,beta,gamma]
    return primitive_atoms,cell_par,inequivalent_pos

### check all

In [153]:
sg = 167
cc = crystals[sg][0]
primitive_atoms,cell_par,inequivalent_pos = get_ibrav5_atoms(cc,sg)
view(cc)
view(primitive_atoms)
print 'sg {}'.format(sg)
print cell_par


sg 167
[9.7049669483204024, 0.0, 0.0, 119.39306125170683, 0.0, 0.0]


In [154]:
symprec = 1e-5
print spg.get_spacegroup(cc,symprec=symprec)
print spg.get_spacegroup(primitive_atoms,symprec=symprec)

R-3c (167)
R-3c (167)


In [147]:
for sg in relevant_sg:
    try:
        cc = crystals[sg][0]
        primitive_atoms,cell_par,inequivalent_pos = get_ibrav5_atoms(cc,sg)
        view(cc)
        view(primitive_atoms)
        print 'sg {}'.format(sg)
        print cell_par
        raw_input("Press Enter to continue...")
    except:
        print 'No structure for sg {}'.format(sg)



sg 146
[3.7820019948990224, 0.0, 0.0, 0.0, 0.0, 0.0]
Press Enter to continue...
sg 148
[5.4039213269642969, 0.0, 0.0, 0.0, 0.0, 0.0]
Press Enter to continue...
sg 155
[4.4237159514123023, 0.0, 0.0, 0.0, 0.0, 0.0]
No structure for sg 155
sg 160
[5.9663080782218865, 0.0, 0.0, 0.0, 0.0, 0.0]
No structure for sg 160
sg 161
[5.4425698931175646, 0.0, 0.0, 0.0, 0.0, 0.0]
No structure for sg 161
sg 166
[10.209972340390793, 0.0, 0.0, 0.0, 0.0, 0.0]
No structure for sg 166
sg 167
[9.7049669483204024, 0.0, 0.0, 0.0, 0.0, 0.0]
No structure for sg 167


### check all

In [15]:
int(spg.get_spacegroup(cc,symprec=1e-3).split(' ')[1][1:-1])

63

In [20]:
sg = 63
cc = crystals[sg][0]
primitive_atoms,cell_par,inequivalent_pos = get_ibrav9_atoms(cc,sg)
view(cc)
view(primitive_atoms)
print 'sg {}'.format(sg)
print cell_par


sg 63
[2.2612070072888382, 2.2612070072888382, 3.6925296780605597, 0.0, 0.0, 0.0]


In [73]:
symprec = 1e-3
print spg.get_spacegroup(cc,symprec=symprec)
print spg.get_spacegroup(primitive_atoms,symprec=symprec)

Cmcm (63)
Cmcm (63)


In [19]:
for sg in relevant_sg:
    try:
        cc = crystals[sg][0]
        
        primitive_atoms,cell_par,inequivalent_pos = get_ibrav9_atoms(cc,sg)
        view(cc)
        view(primitive_atoms)
        print 'sg {}'.format(sg)
        print cell_par
        raw_input("Press Enter to continue...")
    except:
        print 'No structure for sg {}'.format(sg)



sg 20
[2.6649982073776921, 2.6649982073776921, 4.7160166958629146, 0.0, 0.0, 0.0]
Press Enter to continue...
sg 21
[4.1690069584293505, 4.1690069584293505, 3.1600226818471424, 0.0, 0.0, 0.0]
Press Enter to continue...
sg 35
[2.2474941866587579, 2.2474941866587579, 2.1998670905715314, 0.0, 0.0, 0.0]
Press Enter to continue...
sg 36
[3.1447992005676988, 3.1447992005676988, 3.8912346352252323, 0.0, 0.0, 0.0]
Press Enter to continue...
sg 37
[4.5418060211975213, 4.5418060211975213, 2.2821125827075366, 0.0, 0.0, 0.0]
Press Enter to continue...
sg 38
[3.1338742732981419, 3.1338742732981419, 4.432479786791002, 0.0, 0.0, 0.0]
Press Enter to continue...
sg 39
[4.2704818849539832, 4.2704818849539832, 3.8463682540392385, 0.0, 0.0, 0.0]
Press Enter to continue...
sg 40
[2.2523758067186388, 2.2523758067186388, 8.1096037255931535, 0.0, 0.0, 0.0]
Press Enter to continue...
sg 41
[2.2401064064736516, 2.2401064064736516, 7.5060083929890498, 0.0, 0.0, 0.0]
Press Enter to continue...
sg 63
[2.26120700728

# State the inputs for QE

### info to generate automaticaly

In [3]:
species = ['Si',]*2
mass = [28.0855]
PP = ['Si.pbe-n-rrkjus_psl.1.0.0.UPF']
wic = ['4a','16g']
positions = [[0.1],[0.60376669,  0.186657,   0.77821707]]
kpt = [2,2,2]
kpt_offset = [0,0,0]


In [164]:
SGidx = 42
sgInfo = SGTable[SGTable['Table No.'] == 42]
print sgInfo['Crystal System']
sgInfo['Full notation'].values[0][0].encode('utf-8')

210    orthorhombic
Name: Crystal System, dtype: object


'F'

 Bravais-lattice index. If ibrav /= 0, specify EITHER
  [ celldm(1)-celldm(6) ] OR [ A, B, C, cosAB, cosAC, cosBC ]
  but NOT both. The lattice parameter "alat" is set to
  alat = celldm(1) (in a.u.) or alat = A (in Angstrom);
  see below for the other parameters.
  For ibrav=0 specify the lattice vectors in CELL_PARAMETERS,
  optionally the lattice parameter alat = celldm(1) (in a.u.)
  or = A (in Angstrom), or else it is taken from CELL_PARAMETERS

ibrav      structure                   celldm(2)-celldm(6)
                                     or: b,c,cosab,cosac,cosbc
  0          free
      crystal axis provided in input: see card CELL_PARAMETERS

  1          cubic P (sc)
      v1 = a(1,0,0),  v2 = a(0,1,0),  v3 = a(0,0,1)

  2          cubic F (fcc)
      v1 = (a/2)(-1,0,1),  v2 = (a/2)(0,1,1), v3 = (a/2)(-1,1,0)

  3          cubic I (bcc)
      v1 = (a/2)(1,1,1),  v2 = (a/2)(-1,1,1),  v3 = (a/2)(-1,-1,1)

  4          Hexagonal and Trigonal P        celldm(3)=c/a
      v1 = a(1,0,0),  v2 = a(-1/2,sqrt(3)/2,0),  v3 = a(0,0,c/a)

  5          Trigonal R, 3fold axis c        celldm(4)=cos(alpha)
      The crystallographic vectors form a three-fold star around
      the z-axis, the primitive cell is a simple rhombohedron:
      v1 = a(tx,-ty,tz),   v2 = a(0,2ty,tz),   v3 = a(-tx,-ty,tz)
      where c=cos(alpha) is the cosine of the angle alpha between
      any pair of crystallographic vectors, tx, ty, tz are:
        tx=sqrt((1-c)/2), ty=sqrt((1-c)/6), tz=sqrt((1+2c)/3)
 -5          Trigonal R, 3fold axis <111>    celldm(4)=cos(alpha)
      The crystallographic vectors form a three-fold star around
      <111>. Defining a' = a/sqrt(3) :
      v1 = a' (u,v,v),   v2 = a' (v,u,v),   v3 = a' (v,v,u)
      where u and v are defined as
        u = tz - 2*sqrt(2)*ty,  v = tz + sqrt(2)*ty
      and tx, ty, tz as for case ibrav=5
      Note: if you prefer x,y,z as axis in the cubic limit,
            set  u = tz + 2*sqrt(2)*ty,  v = tz - sqrt(2)*ty
            See also the note in Modules/latgen.f90

  6          Tetragonal P (st)               celldm(3)=c/a
      v1 = a(1,0,0),  v2 = a(0,1,0),  v3 = a(0,0,c/a)

  7          Tetragonal I (bct)              celldm(3)=c/a
      v1=(a/2)(1,-1,c/a),  v2=(a/2)(1,1,c/a),  v3=(a/2)(-1,-1,c/a)

  8          Orthorhombic P                  celldm(2)=b/a
                                             celldm(3)=c/a
      v1 = (a,0,0),  v2 = (0,b,0), v3 = (0,0,c)

  9          Orthorhombic base-centered(bco) celldm(2)=b/a
                                             celldm(3)=c/a
      v1 = (a/2, b/2,0),  v2 = (-a/2,b/2,0),  v3 = (0,0,c)
 -9          as 9, alternate description
      v1 = (a/2,-b/2,0),  v2 = (a/2, b/2,0),  v3 = (0,0,c)

 10          Orthorhombic face-centered      celldm(2)=b/a
                                             celldm(3)=c/a
      v1 = (a/2,0,c/2),  v2 = (a/2,b/2,0),  v3 = (0,b/2,c/2)

 11          Orthorhombic body-centered      celldm(2)=b/a
                                             celldm(3)=c/a
      v1=(a/2,b/2,c/2),  v2=(-a/2,b/2,c/2),  v3=(-a/2,-b/2,c/2)

 12          Monoclinic P, unique axis c     celldm(2)=b/a
                                             celldm(3)=c/a,
                                             celldm(4)=cos(ab)
      v1=(a,0,0), v2=(b*cos(gamma),b*sin(gamma),0),  v3 = (0,0,c)
      where gamma is the angle between axis a and b.
-12          Monoclinic P, unique axis b     celldm(2)=b/a
                                             celldm(3)=c/a,
                                             celldm(5)=cos(ac)
      v1 = (a,0,0), v2 = (0,b,0), v3 = (c*cos(beta),0,c*sin(beta))
      where beta is the angle between axis a and c

 13          Monoclinic base-centered        celldm(2)=b/a
                                             celldm(3)=c/a,
                                             celldm(4)=cos(ab)
      v1 = (  a/2,         0,                -c/2),
      v2 = (b*cos(gamma), b*sin(gamma), 0),
      v3 = (  a/2,         0,                  c/2),
      where gamma is the angle between axis a and b

 14          Triclinic                       celldm(2)= b/a,
                                             celldm(3)= c/a,
                                             celldm(4)= cos(bc),
                                             celldm(5)= cos(ac),
                                             celldm(6)= cos(ab)
      v1 = (a, 0, 0),
      v2 = (b*cos(gamma), b*sin(gamma), 0)
      v3 = (c*cos(beta),  c*(cos(alpha)-cos(beta)cos(gamma))/sin(gamma),
           c*sqrt( 1 + 2*cos(alpha)cos(beta)cos(gamma)
                     - cos(alpha)^2-cos(beta)^2-cos(gamma)^2 )/sin(gamma) )
      where alpha is the angle between axis b and c
             beta is the angle between axis a and c
            gamma is the angle between axis a and b

## Load general information

In [159]:
fileNames = {}
fileNames['crystals'] = 'Si-crystals-full-relax.pck'
fileNames['wyck'] = 'SpaceGroup-multiplicity-wickoff-info.pck'
fileNames['general info'] = 'SpaceGroup-general-info.pck'
fileNames['elements info'] = 'General-Info-Elements-fast.pck'

In [160]:
# give the SG 
spaceGroupIdxRange  = range(1,230+1)

Gen  = GenCrystalStructures(spaceGroupIdxRange=spaceGroupIdxRange)
Gen.setCrystals(fileNames['crystals'])
with open(fileNames['wyck'],'rb') as f:
    wyckTable = pck.load(f)
SGTable = pd.read_pickle(fileNames['general info'])
ElemTable = pd.read_pickle(fileNames['elements info'])

## Input for QE

In [186]:
SpaceGroupIdx = 150
zatom = 14
rhocutoff = wfccutoff * 4
wfccutoff = 20
kpt = [2,2,2]
kpt_offset = [0,0,0]


elemInfo = ElemTable[ElemTable['z'] == zatom]

species = [elemInfo['sym'].values[0],]*1
mass = [elemInfo['mass'].values[0]] # in atomic unit
PP = ['Si.pbe-n-rrkjus_psl.1.0.0.UPF']
wyck = [SG2wyckoff(SpaceGroupIdx,wyckTable)]
positions = [SG2Postion(SpaceGroupIdx,Gen.crystals)]
cellParam = SG2cellParam(SpaceGroupIdx,Gen.crystals)
ibrav = SG2ibrav(SpaceGroupIdx,SGTable)
multipicity = wyckTable[SpaceGroupIdx]['Multiplicity'][0]
nvalence = elemInfo['val'].values[0]

if int(0.2*multipicity*nvalence/2.) > 4:
    nbnd = int(multipicity*nvalence/2.+int(0.2*multipicity*nvalence/2.))
else:
    nbnd = int(multipicity*nvalence/2.+4)



In [192]:
print species,mass,nvalence
print cellParam

['Si'] [28.0855] 4
[   3.7128429     3.7128429     8.79360738   90.           90.          120.        ]


## make the QE input block with variable and hard code parts

In [199]:
# define name list of QE
control = {'calculation' : '"scf"',
          'outdir' : '"./out/"',
          'prefix' : '"qe"',
          'pseudo_dir' : '"../pseudo/"',
          'restart_mode' : '"from_scratch"',
          'verbosity' : '"high"',
          'wf_collect' : '.false.',
    }
controlKeys = ['calculation','outdir','prefix','pseudo_dir', 'restart_mode','verbosity' ,'wf_collect']
system = {
          'ecutrho' :   '{:.5f}'.format(rhocutoff),
          'ecutwfc' :   '{:.5f}'.format(wfccutoff),
          'ibrav' : str(ibrav),
          'nat' : str(1),
          'nbnd' : str(nbnd),
          'ntyp' : str(1),
          'occupations' : '"smearing"',
          'smearing' : '"cold"',
          'degauss' :   '1.0000000000d-02',
          'space_group' : str(SpaceGroupIdx),
          'A' : str(cellParam[0]),
          'B' : str(cellParam[1]),
          'C' : str(cellParam[2]),
          'cosAB' : '{:.5f}'.format(np.cos(cellParam[5]*np.pi/180.)),
          'cosAC' : '{:.5f}'.format(np.cos(cellParam[4]*np.pi/180.)),
          'cosBC' : '{:.5f}'.format(np.cos(cellParam[3]*np.pi/180.)),
    }
syskeys = ['degauss', 'ecutrho','ecutwfc','ibrav', 'nat','nbnd','ntyp' ,
           'occupations', 'smearing','space_group','A','B' ,'C', 'cosAB', 'cosAC', 'cosBC' ]
electrons = {'conv_thr':'1.0000000000d-06'}

# define cards of QE
optionkey = 'unit'
atomic_sp = {'ATOMIC_SPECIES':{'species':np.unique(species), 'mass':mass,'PP':PP},'unit':''}
atspkeys = ['species','mass','PP']
atomic_pos = {'ATOMIC_POSITIONS':{'species':species, 'wickoffs':wyck,'positions':positions},'unit':'crystal_sg'}
atposkeys = ['species', 'wickoffs','positions']
kpoints = {'K_POINTS':{'kpoints':kpt+kpt_offset},'unit':'automatic'}
kptkeys = ['kpoints']

In [200]:
qeInput = ''

qeInput += makeNamelist('&CONTROL',control)
qeInput += makeNamelist( '&SYSTEM' ,system,syskeys)
qeInput += makeNamelist( '&ELECTRONS',electrons)
qeInput += makeCard(atomic_sp,cardKeys = atspkeys,optionKey = optionkey,T = True) 
qeInput += makeCard(atomic_pos,cardKeys = atposkeys,optionKey = optionkey,T = True) 
qeInput += makeCard(kpoints,cardKeys = kptkeys,optionKey = optionkey,T = False)
print qeInput

&CONTROL 
  wf_collect = .false. 
  calculation = "scf" 
  verbosity = "high" 
  pseudo_dir = "../pseudo/" 
  prefix = "qe" 
  restart_mode = "from_scratch" 
  outdir = "./out/" 
/ 
&SYSTEM 
  degauss = 1.0000000000d-02 
  ecutrho = 0.01000 
  ecutwfc = 0.08000 
  ibrav = 4 
  nat = 1 
  nbnd = 14 
  ntyp = 1 
  occupations = "smearing" 
  smearing = "cold" 
  space_group = 150 
  A = 3.7128428979 
  B = 3.7128428979 
  C = 8.79360738256 
  cosAB = -0.50000 
  cosAC = 0.00000 
  cosBC = 0.00000 
/ 
&ELECTRONS 
  conv_thr = 1.0000000000d-06 
/ 
ATOMIC_SPECIES   
  Si  28.0855  Si.pbe-n-rrkjus_psl.1.0.0.UPF 
ATOMIC_POSITIONS  crystal_sg 
  Si  6g   0.6666654   0.66666287  0.11896427 
K_POINTS  automatic 
  2  2  2  0  0  0 



# Input file generator function

In [56]:
def makeQEInnput_ibrav0(crystal,ElemTable,
                 zatom = 14,rhocutoff = 20 * 4,wfccutoff = 20,
                        kpt = [2,2,2],kpt_offset = [0,0,0],
                        ppPath='"./pseudo/"'):
    inputDic = {}
    
    elemInfo = ElemTable[ElemTable['z'] == zatom]
    
    mass = [elemInfo['mass'].values[0]] # in atomic unit
    PP = ['Si.pbe-n-rrkjus_psl.1.0.0.UPF']
    nvalence = elemInfo['val'].values[0]
    
    (lattice, positions, numbers) = spg.find_primitive(crystal, symprec=1e-05, angle_tolerance=-1.0)
    primitive_atoms = ase.Atoms(cell=lattice,scaled_positions=positions,numbers=numbers)
    
    positions = primitive_atoms.get_positions()
    cell = primitive_atoms.get_cell()
    ibrav = 0
    Natom = primitive_atoms.get_number_of_atoms()
    species = [elemInfo['sym'].values[0],]*Natom
    
    multiplicity = Natom
    if int(0.2*multiplicity*nvalence/2.) > 4:
        nbnd = int(multiplicity*nvalence/2.+int(0.2*multiplicity*nvalence/2.))
    else:
        nbnd = int(multiplicity*nvalence/2.+4)

    # define name list of QE
    control = {'calculation' : '"scf"',
              'outdir' : '"./out/"',
              'prefix' : '"qe"',
              'pseudo_dir' : ppPath,
              'restart_mode' : '"from_scratch"',
              'verbosity' : '"high"',
              'wf_collect' : '.false.',
        }
    controlKeys = ['calculation','outdir','prefix','pseudo_dir', 'restart_mode','verbosity' ,'wf_collect']
    system = {
              'ecutrho' :   '{:.5f}'.format(rhocutoff),
              'ecutwfc' :   '{:.5f}'.format(wfccutoff),
              'ibrav' : str(ibrav),
              'nat' : str(Natom),
              'nbnd' : str(nbnd),
              'ntyp' : str(1),
              'occupations' : '"smearing"',
              'smearing' : '"cold"',
              'degauss' :   '1.0000000000d-02',
        }
    syskeys = [ 'ecutrho','ecutwfc','ibrav', 'nat','nbnd','ntyp' ,
               'occupations', 'smearing','degauss' ]
    electrons = {'conv_thr':'1.0000000000d-06'}

    # define cards of QE
    optionkey = 'unit'
    atomic_sp = {'ATOMIC_SPECIES':{'species':np.unique(species), 'mass':mass,'PP':PP},'unit':''}
    atspkeys = ['species','mass','PP']
    atomic_pos = {'ATOMIC_POSITIONS':{'species':species,'positions':positions},'unit':'angstrom'}
    atposkeys = ['species','positions']
    kpoints = {'K_POINTS':{'kpoints':kpt+kpt_offset},'unit':'automatic'}
    kptkeys = ['kpoints']
    cell_param = {'CELL_PARAMETERS':{'cell':cell},'unit':'angstrom'}
    cellkeys = ['cell']


    qeInput = ''

    qeInput += makeNamelist('&CONTROL',control)
    qeInput += makeNamelist( '&SYSTEM' ,system,syskeys)
    qeInput += makeNamelist( '&ELECTRONS',electrons)
    qeInput += makeCard(atomic_sp,cardKeys = atspkeys,optionKey = optionkey,T = True) 
    qeInput += makeCard(atomic_pos,cardKeys = atposkeys,optionKey = optionkey,T = True) 
    qeInput += makeCard(kpoints,cardKeys = kptkeys,optionKey = optionkey,T = False)
    qeInput += makeCard(cell_param,cardKeys = cellkeys,optionKey = optionkey,T = True)
    
    inputDic = qeInput
        
    return inputDic

In [61]:
def makeQEInnput_test(crystal,ElemTable,
                 zatom = 14,rhocutoff = 20 * 4,wfccutoff = 20,kpt = [2,2,2],kpt_offset = [0,0,0],ppPath='"./pseudo/"'):
    inputDic = {}
    
    elemInfo = ElemTable[ElemTable['z'] == zatom]
    
    mass = [elemInfo['mass'].values[0]] # in atomic unit
    PP = ['Si.pbe-n-rrkjus_psl.1.0.0.UPF']
    nvalence = elemInfo['val'].values[0]
    
    (lattice, positions, numbers) = spg.find_primitive(crystal, symprec=1e-05, angle_tolerance=-1.0)
    #primitive_atoms = ase.Atoms(cell=lattice,scaled_positions=positions,numbers=numbers)
    primitive_atoms = crystal
    positions = primitive_atoms.get_positions()
    cell = primitive_atoms.get_cell()
    ibrav = 0
    Natom = primitive_atoms.get_number_of_atoms()
    species = [elemInfo['sym'].values[0],]*Natom
    
    multiplicity = Natom
    if int(0.2*multiplicity*nvalence/2.) > 4:
        nbnd = int(multiplicity*nvalence/2.+int(0.2*multiplicity*nvalence/2.))
    else:
        nbnd = int(multiplicity*nvalence/2.+4)

    # define name list of QE
    control = {'calculation' : '"scf"',
              'outdir' : '"./out/"',
              'prefix' : '"qe"',
              'pseudo_dir' : ppPath,
              'restart_mode' : '"from_scratch"',
              'verbosity' : '"high"',
              'wf_collect' : '.false.',
        }
    controlKeys = ['calculation','outdir','prefix','pseudo_dir', 'restart_mode','verbosity' ,'wf_collect']
    system = {
              'ecutrho' :   '{:.5f}'.format(rhocutoff),
              'ecutwfc' :   '{:.5f}'.format(wfccutoff),
              'ibrav' : str(ibrav),
              'nat' : str(Natom),
              'nbnd' : str(nbnd),
              'ntyp' : str(1),
              'occupations' : '"smearing"',
              'smearing' : '"cold"',
              'degauss' :   '1.0000000000d-02',
        }
    syskeys = [ 'ecutrho','ecutwfc','ibrav', 'nat','nbnd','ntyp' ,
               'occupations', 'smearing','degauss' ]
    electrons = {'conv_thr':'1.0000000000d-06'}

    # define cards of QE
    optionkey = 'unit'
    atomic_sp = {'ATOMIC_SPECIES':{'species':np.unique(species), 'mass':mass,'PP':PP},'unit':''}
    atspkeys = ['species','mass','PP']
    atomic_pos = {'ATOMIC_POSITIONS':{'species':species,'positions':positions},'unit':'angstrom'}
    atposkeys = ['species','positions']
    kpoints = {'K_POINTS':{'kpoints':kpt+kpt_offset},'unit':'automatic'}
    kptkeys = ['kpoints']
    cell_param = {'CELL_PARAMETERS':{'cell':cell},'unit':'angstrom'}
    cellkeys = ['cell']


    qeInput = ''

    qeInput += makeNamelist('&CONTROL',control)
    qeInput += makeNamelist( '&SYSTEM' ,system,syskeys)
    qeInput += makeNamelist( '&ELECTRONS',electrons)
    qeInput += makeCard(atomic_sp,cardKeys = atspkeys,optionKey = optionkey,T = True) 
    qeInput += makeCard(atomic_pos,cardKeys = atposkeys,optionKey = optionkey,T = True) 
    qeInput += makeCard(kpoints,cardKeys = kptkeys,optionKey = optionkey,T = False)
    qeInput += makeCard(cell_param,cardKeys = cellkeys,optionKey = optionkey,T = True)
    
    inputDic = qeInput
        
    return inputDic

## test

In [62]:
cc = crystals[229][0]
inp = makeQEInnput_test(cc,ElemTable,
                 zatom = 14,rhocutoff = 20 * 4,wfccutoff = 20,kpt = [2,2,2],kpt_offset = [0,0,0])
print inp

&CONTROL 
  wf_collect = .false. 
  calculation = "scf" 
  verbosity = "high" 
  pseudo_dir = "./pseudo/" 
  prefix = "qe" 
  restart_mode = "from_scratch" 
  outdir = "./out/" 
/ 
&SYSTEM 
  ecutrho = 80.00000 
  ecutwfc = 20.00000 
  ibrav = 0 
  nat = 96 
  nbnd = 230 
  ntyp = 1 
  occupations = "smearing" 
  smearing = "cold" 
  degauss = 1.0000000000d-02 
/ 
&ELECTRONS 
  conv_thr = 1.0000000000d-06 
/ 
ATOMIC_SPECIES   
  Si  28.0855  Si.pbe-n-rrkjus_psl.1.0.0.UPF 
ATOMIC_POSITIONS  angstrom 
  Si    9.50434336  11.10461932   5.63759121 
  Si   4.03831459  2.43803863  5.63759121 
  Si    4.03831459  11.10461932   7.90506674 
  Si   9.50434336  2.43803863  7.90506674 
  Si    5.63759121   9.50434336  11.10461932 
  Si   5.63759121  4.03831459  2.43803863 
  Si    7.90506674   4.03831459  11.10461932 
  Si   7.90506674  9.50434336  2.43803863 
  Si   11.10461932   5.63759121   9.50434336 
  Si   2.43803863  5.63759121  4.03831459 
  Si   11.10461932   7.90506674   4.03831459 
  Si

## Load general information

In [48]:
fileNames = {}
dataPath = '../'  
fileNames['crystals'] = dataPath+ 'partial_input_crystals_sg3-230.pck'
fileNames['wyck'] = dataPath+'SpaceGroup-multiplicity-wickoff-info.pck'
fileNames['general info'] = dataPath+'SpaceGroup-general-info.pck'
fileNames['elements info'] = dataPath+'General-Info-Elements-fast.pck'

In [49]:
# give the SG 
spaceGroupIdxRange  = range(1,230+1)

with open(fileNames['crystals'],'rb') as f:
    crystals = pck.load(f)
    
with open(fileNames['wyck'],'rb') as f:
    WyckTable = pck.load(f)
SGTable = pd.read_pickle(fileNames['general info'])
ElemTable = pd.read_pickle(fileNames['elements info'])

In [50]:
sg2ibrav = {it:SG2ibrav(it,SGTable) for it in range(1,230+1)}
# print sg2ibrav
relevant_sg = []
for key,item in sg2ibrav.iteritems():
    if item == 3:
        relevant_sg.append(key)
print relevant_sg

[197, 199, 204, 206, 211, 214, 217, 220, 229, 230]


In [51]:
inputDic = {}
for sg in relevant_sg:
    inputDic[sg] = makeQEInnput_ibrav0(crystals[sg][0],ElemTable,
                 zatom = 14,rhocutoff = 20 * 4,wfccutoff = 20,kpt = [2,2,2],kpt_offset = [0,0,0],ppPath='"./pseudo/"')

In [52]:
def make_dir(file_path):
    directory = os.path.abspath(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

In [58]:
#'/Users/iMac/Documents/Workdir/QMAT/check-QE-crystal_sg'
dirNames = {sg:'./data_ibrav3/sg-test_'+str(sg) for sg in relevant_sg}
# dirNames = {sg:'./data_ibrav3/sg-test_'+str(sg) for sg in [229]}

In [59]:
import subprocess as sp
os.environ['TMPDIR'] = "/tmp/"

In [60]:
exitState = {}
path2pw = '/home/musil/source/qe-6.1/bin/'
for sg in dirNames:
    make_dir(dirNames[sg])
    inputName = os.path.abspath(dirNames[sg]+'/qe.in')
    outputName = os.path.abspath(dirNames[sg]+'/qe.out')
    errName = os.path.abspath(dirNames[sg]+'/qe.err')
    with open(inputName,'w') as f:
        f.write(inputDic[sg])
    param = '-in '+ inputName
    print param
    # Set up the echo command and direct the output to a pipe
    exitState[sg] = sp.call('mpirun -np 2 '+path2pw+'pw.x ' + param , stdout=open(outputName, 'w'), 
                             stderr=open(errName, 'w'), shell=True)
    if exitState[sg]:
        print sg


-in /local/Dropbox/Felix_work/qmat/qe-sg-detection-test/data_ibrav3/sg-test_197/qe.in
-in /local/Dropbox/Felix_work/qmat/qe-sg-detection-test/data_ibrav3/sg-test_230/qe.in
-in /local/Dropbox/Felix_work/qmat/qe-sg-detection-test/data_ibrav3/sg-test_199/qe.in
-in /local/Dropbox/Felix_work/qmat/qe-sg-detection-test/data_ibrav3/sg-test_204/qe.in
-in /local/Dropbox/Felix_work/qmat/qe-sg-detection-test/data_ibrav3/sg-test_206/qe.in
-in /local/Dropbox/Felix_work/qmat/qe-sg-detection-test/data_ibrav3/sg-test_211/qe.in
-in /local/Dropbox/Felix_work/qmat/qe-sg-detection-test/data_ibrav3/sg-test_214/qe.in
-in /local/Dropbox/Felix_work/qmat/qe-sg-detection-test/data_ibrav3/sg-test_217/qe.in
-in /local/Dropbox/Felix_work/qmat/qe-sg-detection-test/data_ibrav3/sg-test_220/qe.in
-in /local/Dropbox/Felix_work/qmat/qe-sg-detection-test/data_ibrav3/sg-test_229/qe.in


In [ ]:
prbSg = []
for sg in exitState:
    if exitState[sg]:
        prbSg.append(sg)
        
print prbSg

In [65]:
print relevant_sg

[197, 199, 204, 206, 211, 214, 217, 220, 229, 230]


In [82]:
db = spg.get_symmetry(crystals[206][0])
syms = [(r,t) for r,t in zip(db['rotations'],db['translations'])]

In [83]:
len(syms)

48

In [79]:
syms[5:10]

[(array([[ 0, -1,  0],
         [ 0,  0,  1],
         [-1,  0,  0]], dtype=int32), array([ 0.5,  1. ,  1. ])),
 (array([[ 0, -1,  0],
         [ 0,  0, -1],
         [ 1,  0,  0]], dtype=int32), array([ 0.5,  1. ,  0.5])),
 (array([[0, 1, 0],
         [0, 0, 1],
         [1, 0, 0]], dtype=int32), array([ 0.5,  0.5,  0.5])),
 (array([[ 0,  0, -1],
         [ 1,  0,  0],
         [ 0, -1,  0]], dtype=int32), array([ 0.5,  1. ,  1. ])),
 (array([[0, 0, 1],
         [1, 0, 0],
         [0, 1, 0]], dtype=int32), array([ 0.5,  0.5,  0.5]))]